# Procesamiento de la data del test

In [891]:
import numpy as np
import pandas as pd
import cv2
import requests
from io import BytesIO
from PIL import Image
from skimage import color, feature
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import os

Obtenemos el test y las columas necesarias para sacar las imagenes

In [892]:
test_path = "../data/test/movies_test.csv"
df_test = pd.read_csv(test_path)
df_test

,movieId,title
0,45635,"Notorious Bettie Page, The (2005)"
1,1373,Star Trek V: The Final Frontier (1989)
2,7325,Starsky & Hutch (2004)
3,389,"Colonel Chabert, Le (1994)"
4,8920,"Country Girl, The (1954)"
...,...,...
2918,25833,Camille (1936)
2919,156553,Zoom (2015)
2920,136020,Spectre (2015)
2921,2616,Dick Tracy (1990)


In [893]:
dub = df_test['movieId'].duplicated().any()
dub

np.False_

In [894]:
links_path = "../data/ml-25m/links.csv"
df_links = pd.read_csv(links_path)
df_links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
62418,209157,6671244,499546.0
62419,209159,297986,63407.0
62420,209163,6755366,553036.0
62421,209169,249603,162892.0


In [895]:
dub = df_links['movieId'].duplicated().any()
dub

np.False_

In [896]:
df_test_links = pd.merge(df_test, df_links, how='left', on='movieId')
df_test_links

,movieId,title,imdbId,tmdbId
0,45635,"Notorious Bettie Page, The (2005)",404802.0,15402.0
1,1373,Star Trek V: The Final Frontier (1989),98382.0,172.0
2,7325,Starsky & Hutch (2004),335438.0,9384.0
3,389,"Colonel Chabert, Le (1994)",109454.0,41580.0
4,8920,"Country Girl, The (1954)",46874.0,2438.0
...,...,...,...,...
2918,25833,Camille (1936),28683.0,43874.0
2919,156553,Zoom (2015),3763866.0,351065.0
2920,136020,Spectre (2015),2379713.0,206647.0
2921,2616,Dick Tracy (1990),99422.0,8592.0


In [897]:
df_test_links.isna().sum()

movieId     0
title       0
imdbId     88
tmdbId     91
dtype: int64

In [898]:
df_test_links = df_test_links.dropna()

In [899]:
df_test_links.isna().sum()

movieId    0
title      0
imdbId     0
tmdbId     0
dtype: int64

In [900]:
df_test_links

,movieId,title,imdbId,tmdbId
0,45635,"Notorious Bettie Page, The (2005)",404802.0,15402.0
1,1373,Star Trek V: The Final Frontier (1989),98382.0,172.0
2,7325,Starsky & Hutch (2004),335438.0,9384.0
3,389,"Colonel Chabert, Le (1994)",109454.0,41580.0
4,8920,"Country Girl, The (1954)",46874.0,2438.0
...,...,...,...,...
2918,25833,Camille (1936),28683.0,43874.0
2919,156553,Zoom (2015),3763866.0,351065.0
2920,136020,Spectre (2015),2379713.0,206647.0
2921,2616,Dick Tracy (1990),99422.0,8592.0


In [901]:
df_imdb = pd.read_csv("../data/archive/MovieGenre.csv", encoding='latin1')
df_imdb

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...
40103,83168,http://www.imdb.com/title/tt83168,Tanya's Island (1980),4.3,Drama,https://images-na.ssl-images-amazon.com/images...
40104,82875,http://www.imdb.com/title/tt82875,Pacific Banana (1981),4.7,Comedy,https://images-na.ssl-images-amazon.com/images...
40105,815258,http://www.imdb.com/title/tt815258,Werewolf in a Womens Prison (2006),4.5,Horror,https://images-na.ssl-images-amazon.com/images...
40106,79142,http://www.imdb.com/title/tt79142,Xiao zi ming da (1979),6.5,Action|Comedy,https://images-na.ssl-images-amazon.com/images...


In [902]:
df_test_links_imgs = pd.merge(df_test_links, df_imdb, how='left', on='imdbId')
df_test_links_imgs

,movieId,title,imdbId,tmdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,45635,"Notorious Bettie Page, The (2005)",404802.0,15402.0,http://www.imdb.com/title/tt404802,The Notorious Bettie Page (2005),6.6,Biography|Drama,https://images-na.ssl-images-amazon.com/images...
1,1373,Star Trek V: The Final Frontier (1989),98382.0,172.0,http://www.imdb.com/title/tt98382,Star Trek V: The Final Frontier (1989),5.4,Action|Adventure|Fantasy,https://images-na.ssl-images-amazon.com/images...
2,7325,Starsky & Hutch (2004),335438.0,9384.0,http://www.imdb.com/title/tt335438,Starsky & Hutch (2004),6.1,Comedy|Crime,https://images-na.ssl-images-amazon.com/images...
3,389,"Colonel Chabert, Le (1994)",109454.0,41580.0,http://www.imdb.com/title/tt109454,Colonel Chabert (1994),7.0,Drama|History|Romance,https://images-na.ssl-images-amazon.com/images...
4,8920,"Country Girl, The (1954)",46874.0,2438.0,http://www.imdb.com/title/tt46874,The Country Girl (1954),7.3,Drama|Music,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...,...,...,...
2868,25833,Camille (1936),28683.0,43874.0,http://www.imdb.com/title/tt28683,Camille (1936),7.5,Drama|Romance,https://images-na.ssl-images-amazon.com/images...
2869,156553,Zoom (2015),3763866.0,351065.0,http://www.imdb.com/title/tt3763866,Zoom (2015),6.2,Animation|Comedy|Drama,https://images-na.ssl-images-amazon.com/images...
2870,136020,Spectre (2015),2379713.0,206647.0,http://www.imdb.com/title/tt2379713,Spectre (2015),6.8,Action|Adventure|Thriller,https://images-na.ssl-images-amazon.com/images...
2871,2616,Dick Tracy (1990),99422.0,8592.0,http://www.imdb.com/title/tt99422,Dick Tracy (1990),6.1,Action|Comedy|Crime,https://images-na.ssl-images-amazon.com/images...


La cantidad aumento, esto nos indica la precensia de duplicados

In [903]:
df_test_links_imgs['movieId'].duplicated().sum()

np.int64(41)

In [904]:
df_test_links_imgs = df_test_links_imgs.drop_duplicates(subset='movieId', keep='first')
df_test_links_imgs

,movieId,title,imdbId,tmdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,45635,"Notorious Bettie Page, The (2005)",404802.0,15402.0,http://www.imdb.com/title/tt404802,The Notorious Bettie Page (2005),6.6,Biography|Drama,https://images-na.ssl-images-amazon.com/images...
1,1373,Star Trek V: The Final Frontier (1989),98382.0,172.0,http://www.imdb.com/title/tt98382,Star Trek V: The Final Frontier (1989),5.4,Action|Adventure|Fantasy,https://images-na.ssl-images-amazon.com/images...
2,7325,Starsky & Hutch (2004),335438.0,9384.0,http://www.imdb.com/title/tt335438,Starsky & Hutch (2004),6.1,Comedy|Crime,https://images-na.ssl-images-amazon.com/images...
3,389,"Colonel Chabert, Le (1994)",109454.0,41580.0,http://www.imdb.com/title/tt109454,Colonel Chabert (1994),7.0,Drama|History|Romance,https://images-na.ssl-images-amazon.com/images...
4,8920,"Country Girl, The (1954)",46874.0,2438.0,http://www.imdb.com/title/tt46874,The Country Girl (1954),7.3,Drama|Music,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...,...,...,...
2868,25833,Camille (1936),28683.0,43874.0,http://www.imdb.com/title/tt28683,Camille (1936),7.5,Drama|Romance,https://images-na.ssl-images-amazon.com/images...
2869,156553,Zoom (2015),3763866.0,351065.0,http://www.imdb.com/title/tt3763866,Zoom (2015),6.2,Animation|Comedy|Drama,https://images-na.ssl-images-amazon.com/images...
2870,136020,Spectre (2015),2379713.0,206647.0,http://www.imdb.com/title/tt2379713,Spectre (2015),6.8,Action|Adventure|Thriller,https://images-na.ssl-images-amazon.com/images...
2871,2616,Dick Tracy (1990),99422.0,8592.0,http://www.imdb.com/title/tt99422,Dick Tracy (1990),6.1,Action|Comedy|Crime,https://images-na.ssl-images-amazon.com/images...


Verificamos los nulos

In [905]:
df_test_links_imgs.isna().sum()

movieId         0
title           0
imdbId          0
tmdbId          0
Imdb Link     115
Title         115
IMDB Score    115
Genre         115
Poster        116
dtype: int64

In [906]:
df_test_links_imgs = df_test_links_imgs.dropna()
df_test_links_imgs.isna().sum()

movieId       0
title         0
imdbId        0
tmdbId        0
Imdb Link     0
Title         0
IMDB Score    0
Genre         0
Poster        0
dtype: int64

In [907]:
df_test_links_imgs

,movieId,title,imdbId,tmdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,45635,"Notorious Bettie Page, The (2005)",404802.0,15402.0,http://www.imdb.com/title/tt404802,The Notorious Bettie Page (2005),6.6,Biography|Drama,https://images-na.ssl-images-amazon.com/images...
1,1373,Star Trek V: The Final Frontier (1989),98382.0,172.0,http://www.imdb.com/title/tt98382,Star Trek V: The Final Frontier (1989),5.4,Action|Adventure|Fantasy,https://images-na.ssl-images-amazon.com/images...
2,7325,Starsky & Hutch (2004),335438.0,9384.0,http://www.imdb.com/title/tt335438,Starsky & Hutch (2004),6.1,Comedy|Crime,https://images-na.ssl-images-amazon.com/images...
3,389,"Colonel Chabert, Le (1994)",109454.0,41580.0,http://www.imdb.com/title/tt109454,Colonel Chabert (1994),7.0,Drama|History|Romance,https://images-na.ssl-images-amazon.com/images...
4,8920,"Country Girl, The (1954)",46874.0,2438.0,http://www.imdb.com/title/tt46874,The Country Girl (1954),7.3,Drama|Music,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...,...,...,...
2868,25833,Camille (1936),28683.0,43874.0,http://www.imdb.com/title/tt28683,Camille (1936),7.5,Drama|Romance,https://images-na.ssl-images-amazon.com/images...
2869,156553,Zoom (2015),3763866.0,351065.0,http://www.imdb.com/title/tt3763866,Zoom (2015),6.2,Animation|Comedy|Drama,https://images-na.ssl-images-amazon.com/images...
2870,136020,Spectre (2015),2379713.0,206647.0,http://www.imdb.com/title/tt2379713,Spectre (2015),6.8,Action|Adventure|Thriller,https://images-na.ssl-images-amazon.com/images...
2871,2616,Dick Tracy (1990),99422.0,8592.0,http://www.imdb.com/title/tt99422,Dick Tracy (1990),6.1,Action|Comedy|Crime,https://images-na.ssl-images-amazon.com/images...


In [908]:
df_imgs_extractor = df_test_links_imgs[["movieId", "Poster"]]
df_imgs_extractor

,movieId,Poster
0,45635,https://images-na.ssl-images-amazon.com/images...
1,1373,https://images-na.ssl-images-amazon.com/images...
2,7325,https://images-na.ssl-images-amazon.com/images...
3,389,https://images-na.ssl-images-amazon.com/images...
4,8920,https://images-na.ssl-images-amazon.com/images...
...,...,...
2868,25833,https://images-na.ssl-images-amazon.com/images...
2869,156553,https://images-na.ssl-images-amazon.com/images...
2870,136020,https://images-na.ssl-images-amazon.com/images...
2871,2616,https://images-na.ssl-images-amazon.com/images...


In [909]:
df_genere_extractor = df_test_links_imgs[["movieId", "Genre"]]
df_genere_extractor

,movieId,Genre
0,45635,Biography|Drama
1,1373,Action|Adventure|Fantasy
2,7325,Comedy|Crime
3,389,Drama|History|Romance
4,8920,Drama|Music
...,...,...
2868,25833,Drama|Romance
2869,156553,Animation|Comedy|Drama
2870,136020,Action|Adventure|Thriller
2871,2616,Action|Comedy|Crime


In [910]:
# One-hot encoding de la columna 'genres'
genre_dummies = df_genere_extractor['Genre'].str.get_dummies(sep='|')

# Unimos las dummies al DataFrame original
df_genere_extractor = df_genere_extractor.join(genre_dummies)

df_genere_extractor = df_genere_extractor.drop('Genre', axis=1)

display(df_genere_extractor)

,movieId,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,Mystery,News,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,45635,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1373,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7325,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,389,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,8920,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2868,25833,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2869,156553,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2870,136020,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2871,2616,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [911]:
df_genere_extractor_d = df_genere_extractor.drop_duplicates(subset='movieId')
df_genere_extractor_d

,movieId,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,Mystery,News,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,45635,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1373,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7325,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,389,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,8920,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2868,25833,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2869,156553,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2870,136020,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2871,2616,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Extraccion de Features visuales

In [912]:
# Historgama de Color
def extract_hsv(img, bins=32):
    hsv = color.rgb2hsv(img)
    h = np.histogram(hsv[:, :, 0], bins=bins, range=(0, 1))[0]
    s = np.histogram(hsv[:, :, 1], bins=bins, range=(0, 1))[0]
    v = np.histogram(hsv[:, :, 2], bins=bins, range=(0, 1))[0]
    return np.concatenate([h, s, v])

# Descriptores de Textura
def extract_lbp(img, P=8, R=1):
    g = color.rgb2gray(img)
    g_uint8 = (g * 255).astype(np.uint8)
    lbp = feature.local_binary_pattern(g_uint8, P, R, method='uniform')
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp, bins=n_bins, range=(0, n_bins), density=True)
    return hist

# Hu Moments
def extract_hu(img):
    gray = color.rgb2gray(img)
    gray_uint8 = (gray * 255).astype(np.uint8)
    moments = cv2.moments(gray_uint8)
    hu = cv2.HuMoments(moments).flatten()
    return hu

# Extractor paralelizado
def extract_all_features(img):
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_hsv = executor.submit(extract_hsv, img)
        future_lbp = executor.submit(extract_lbp, img)
        future_hu = executor.submit(extract_hu, img)
        
        hsv = future_hsv.result()
        lbp = future_lbp.result()
        hu = future_hu.result()
        
        return np.concatenate([hsv, lbp, hu])

In [815]:
# Funcion de descarga de imgs
def download_img(url):
    try:
        response = requests.get(url, timeout=5)
        img = Image.open(BytesIO(response.content)).convert("RGB")
        return np.array(img)
    except Exception:
        return None

In [816]:
# Funcion de procesamiento por fila
def process_row(row, id_col, url_col):
    movie_id = row[id_col]
    url = row[url_col]
    img = download_img(url)
    if img is None:
        return None
    try:
        features = extract_all_features(img)
        return (movie_id, features)
    except Exception:
        return None

In [817]:
# Funcion principal que arma el dataset de features
def extract_features_from_dataframe(df, id, url):
    num_workers = max(1, min(32, (os.cpu_count() or 4) - 1))
    print(f"Usando {num_workers} hilos...")

    # Procesamiento de imgs en paralelo
    results = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        fs = [executor.submit(process_row, row, id, url) for _, row in df.iterrows()]
        for f in tqdm(as_completed(fs), total=len(fs)):
            result = f.result()
            if result:
                results.append(result)

    # Construir dataframe final
    ids = [r[0] for r in results]
    vec = [r[1] for r in results]
    features_df = pd.DataFrame(vec)
    features_df.insert(0, id, ids)

    return features_df

In [913]:
df_imgs_features = extract_features_from_dataframe(df_imgs_extractor, id="movieId", url="Poster")
df_imgs_features

Usando 7 hilos...


100%|██████████| 2716/2716 [00:55<00:00, 49.06it/s]


,movieId,0,1,2,3,4,5,6,7,8,...,103,104,105,106,107,108,109,110,111,112
0,141994,5196.0,2374.0,3968.0,3703.0,2795.0,2252.0,1144.0,524.0,578.0,...,0.102079,0.121474,0.195875,0.001201,1.406197e-07,2.476164e-11,1.321669e-11,2.380754e-22,4.909406e-15,-2.207549e-23
1,389,14316.0,12024.0,5204.0,1361.0,617.0,755.0,573.0,425.0,220.0,...,0.085247,0.134800,0.114667,0.002331,2.463068e-07,2.112036e-10,7.919602e-10,-5.472358e-20,3.915598e-13,-3.192397e-19
2,93040,27903.0,9873.0,635.0,122.0,42.0,162.0,15.0,9.0,56.0,...,0.056319,0.315914,0.110382,0.002082,4.497876e-07,6.595129e-10,4.500493e-10,2.411771e-19,2.831688e-13,4.417557e-20
3,7325,14449.0,4762.0,4265.0,2567.0,1030.0,455.0,156.0,57.0,97.0,...,0.075857,0.274459,0.143800,0.001395,2.607370e-07,1.777852e-11,2.348040e-13,-3.302922e-25,1.001093e-16,-3.479332e-25
4,8920,4337.0,920.0,518.0,790.0,11560.0,7018.0,615.0,113.0,95.0,...,0.066262,0.440237,0.093366,0.002354,8.109859e-07,3.010085e-12,2.227391e-11,1.633792e-22,2.004052e-14,8.106013e-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,103,1318.0,682.0,241.0,65.0,24.0,75.0,35.0,32.0,32.0,...,0.076082,0.277227,0.095600,0.001935,8.271632e-07,2.617360e-10,3.756313e-10,-2.495480e-20,-2.262682e-13,1.151067e-19
2139,27584,13535.0,1248.0,527.0,307.0,118.0,323.0,64.0,33.0,67.0,...,0.075262,0.330367,0.125554,0.004143,5.538892e-06,1.279362e-09,1.610004e-11,-1.871430e-21,-3.128895e-14,-1.355334e-21
2140,156553,916.0,2742.0,4241.0,4614.0,7287.0,1819.0,885.0,377.0,306.0,...,0.091766,0.138449,0.147183,0.001210,7.026354e-08,6.073586e-11,3.056808e-11,7.245548e-22,6.886263e-15,-1.099919e-21
2141,136020,1170.0,780.0,150.0,208.0,202.0,320.0,179.0,55.0,114.0,...,0.075365,0.322167,0.093776,0.001501,2.839812e-07,3.010687e-11,5.652909e-11,2.330807e-21,2.995776e-14,-7.661175e-23


In [914]:
df_imgs_features

,movieId,0,1,2,3,4,5,6,7,8,...,103,104,105,106,107,108,109,110,111,112
0,141994,5196.0,2374.0,3968.0,3703.0,2795.0,2252.0,1144.0,524.0,578.0,...,0.102079,0.121474,0.195875,0.001201,1.406197e-07,2.476164e-11,1.321669e-11,2.380754e-22,4.909406e-15,-2.207549e-23
1,389,14316.0,12024.0,5204.0,1361.0,617.0,755.0,573.0,425.0,220.0,...,0.085247,0.134800,0.114667,0.002331,2.463068e-07,2.112036e-10,7.919602e-10,-5.472358e-20,3.915598e-13,-3.192397e-19
2,93040,27903.0,9873.0,635.0,122.0,42.0,162.0,15.0,9.0,56.0,...,0.056319,0.315914,0.110382,0.002082,4.497876e-07,6.595129e-10,4.500493e-10,2.411771e-19,2.831688e-13,4.417557e-20
3,7325,14449.0,4762.0,4265.0,2567.0,1030.0,455.0,156.0,57.0,97.0,...,0.075857,0.274459,0.143800,0.001395,2.607370e-07,1.777852e-11,2.348040e-13,-3.302922e-25,1.001093e-16,-3.479332e-25
4,8920,4337.0,920.0,518.0,790.0,11560.0,7018.0,615.0,113.0,95.0,...,0.066262,0.440237,0.093366,0.002354,8.109859e-07,3.010085e-12,2.227391e-11,1.633792e-22,2.004052e-14,8.106013e-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,103,1318.0,682.0,241.0,65.0,24.0,75.0,35.0,32.0,32.0,...,0.076082,0.277227,0.095600,0.001935,8.271632e-07,2.617360e-10,3.756313e-10,-2.495480e-20,-2.262682e-13,1.151067e-19
2139,27584,13535.0,1248.0,527.0,307.0,118.0,323.0,64.0,33.0,67.0,...,0.075262,0.330367,0.125554,0.004143,5.538892e-06,1.279362e-09,1.610004e-11,-1.871430e-21,-3.128895e-14,-1.355334e-21
2140,156553,916.0,2742.0,4241.0,4614.0,7287.0,1819.0,885.0,377.0,306.0,...,0.091766,0.138449,0.147183,0.001210,7.026354e-08,6.073586e-11,3.056808e-11,7.245548e-22,6.886263e-15,-1.099919e-21
2141,136020,1170.0,780.0,150.0,208.0,202.0,320.0,179.0,55.0,114.0,...,0.075365,0.322167,0.093776,0.001501,2.839812e-07,3.010687e-11,5.652909e-11,2.330807e-21,2.995776e-14,-7.661175e-23


Ahora buscamos los faltantes con la API

In [915]:
df_test_faltantes = df_test_links_imgs[~df_test_links_imgs['movieId'].isin(df_imgs_features['movieId'])]
df_test_faltantes

,movieId,title,imdbId,tmdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,45635,"Notorious Bettie Page, The (2005)",404802.0,15402.0,http://www.imdb.com/title/tt404802,The Notorious Bettie Page (2005),6.6,Biography|Drama,https://images-na.ssl-images-amazon.com/images...
1,1373,Star Trek V: The Final Frontier (1989),98382.0,172.0,http://www.imdb.com/title/tt98382,Star Trek V: The Final Frontier (1989),5.4,Action|Adventure|Fantasy,https://images-na.ssl-images-amazon.com/images...
8,1446,Kolya (Kolja) (1996),116790.0,784.0,http://www.imdb.com/title/tt116790,Kolya (1996),7.8,Comedy|Drama|Music,https://images-na.ssl-images-amazon.com/images...
13,423,Blown Away (1994),109303.0,178.0,http://www.imdb.com/title/tt109303,Blown Away (1994),6.2,Action|Crime|Drama,https://images-na.ssl-images-amazon.com/images...
21,6625,Camp Nowhere (1994),109369.0,17745.0,http://www.imdb.com/title/tt109369,Camp Nowhere (1994),5.9,Adventure|Comedy|Family,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...,...,...,...
2839,80162,"Horde, The (La Horde) (2009)",1183276.0,37905.0,http://www.imdb.com/title/tt1183276,La horde (2009),5.9,Action|Horror,https://images-na.ssl-images-amazon.com/images...
2865,163653,David Cross: Making America Great Again (2016),5943936.0,408546.0,http://www.imdb.com/title/tt5943936,David Cross: Making America Great Again (2016),6.7,Comedy,https://images-na.ssl-images-amazon.com/images...
2868,25833,Camille (1936),28683.0,43874.0,http://www.imdb.com/title/tt28683,Camille (1936),7.5,Drama|Romance,https://images-na.ssl-images-amazon.com/images...
2871,2616,Dick Tracy (1990),99422.0,8592.0,http://www.imdb.com/title/tt99422,Dick Tracy (1990),6.1,Action|Comedy|Crime,https://images-na.ssl-images-amazon.com/images...


In [916]:
from dotenv import load_dotenv
import requests
import os

load_dotenv()

TMDB_API_KEY = os.getenv("API_KEY_TMBD") # Key para usar la API
# print(TMDB_API_KEY[0:5]) # Comprobar si la llave es correcta

BASE_URL = 'https://api.themoviedb.org/3' # Base para generar las URLs de las imagenes
FALLBACK = 'https://image.tmdb.org/t/p/' # Falback si falla la config
DEFAULT_SIZE = 'original' # Tamaño por defecto en caso no encuentre el poster del tamaño pedio

# Variable global para almacenar la configuración de imágenes (para no llamarla siempre)
tmdb_image_config = None

# Funcion para obtener la configuracion de la API de imagenes de TMBd
def get_config():
    global tmdb_image_config
    if tmdb_image_config is not None:
        return tmdb_image_config

    if not TMDB_API_KEY:
        print("Advertencia: API Key de TMDb no configurada. No se puede obtener la configuración.")
        return None

    config_url = f"{BASE_URL}/configuration?api_key={TMDB_API_KEY}"
    try:
        response = requests.get(config_url, timeout=5)
        response.raise_for_status()
        tmdb_image_config = response.json().get('images', {})
        print("Configuración de imágenes de TMDb obtenida exitosamente.")
        return tmdb_image_config
    except requests.exceptions.RequestException as e:
        print(f"Error obteniendo configuración de TMDb: {e}")
        return None

# Constructor de URLs para posters
def url_builder(path: str, size: str = DEFAULT_SIZE):
    cfg = get_config() or {}
    base = cfg.get('secure_base_url', FALLBACK)
    sizes = cfg.get('poster_sizes', [])

    chosen_size = size if size in sizes else DEFAULT_SIZE # En caso no se encuentre el tamaño indicado, toma el size original
    return f"{base}{chosen_size}{path}"

# Funcion para obtener la URL del poster usando el TMBdID
def get_poster_by_id(movie_id: int, size: str = DEFAULT_SIZE, language: bool = 0):
    if language:
        url = f"{BASE_URL}/movie/{movie_id}?api_key={TMDB_API_KEY}&language=es-ES"
    else:
        url = f"{BASE_URL}/movie/{movie_id}?api_key={TMDB_API_KEY}"
    try:
        res = requests.get(url, timeout=30)
        res.raise_for_status()
        data = res.json()
        p_path = data.get('poster_path')
        if not p_path:
            return None
        return url_builder(p_path, size)
    except Exception as e:
        print(f"Error al obtener poster para ID {movie_id}: {e}")
        return None

# Funcion para obtener la URL del poster usando el titulo y año
def get_poster_by_title(title: str, year: int = None, size: str = DEFAULT_SIZE, language: bool = 0):
    if language:
        params = {
            'api_key': TMDB_API_KEY,
            'query': title,
            'language': 'es-ES',
        }
    else:
        params = {
            'api_key': TMDB_API_KEY,
            'query': title
        }
    if year:
        params['year'] = year
    try:
        url = f"{BASE_URL}/search/movie"
        res = requests.get(url, params=params, timeout=5)
        res.raise_for_status()
        data = res.json().get('results', [])
        if not data:
            return None
        f = data[0]
        p_path = f.get('poster_path')
        if not p_path:
            return None
        return url_builder(p_path, size)
    except Exception as e:
        print(f"Error al obtener poster para la pelicula {title}: {e}")
        return None


In [917]:
get_config()

Configuración de imágenes de TMDb obtenida exitosamente.


{'base_url': 'http://image.tmdb.org/t/p/',
 'secure_base_url': 'https://image.tmdb.org/t/p/',
 'backdrop_sizes': ['w300', 'w780', 'w1280', 'original'],
 'logo_sizes': ['w45', 'w92', 'w154', 'w185', 'w300', 'w500', 'original'],
 'poster_sizes': ['w92', 'w154', 'w185', 'w342', 'w500', 'w780', 'original'],
 'profile_sizes': ['w45', 'w185', 'h632', 'original'],
 'still_sizes': ['w92', 'w185', 'w300', 'original']}

In [918]:
df_test_faltantes['Poster'] = None
df_test_faltantes

/var/folders/2w/09s3w8g13ljfzp1b6hpsfjym0000gn/T/ipykernel_1885/4038259082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_faltantes['Poster'] = None


,movieId,title,imdbId,tmdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,45635,"Notorious Bettie Page, The (2005)",404802.0,15402.0,http://www.imdb.com/title/tt404802,The Notorious Bettie Page (2005),6.6,Biography|Drama,None
1,1373,Star Trek V: The Final Frontier (1989),98382.0,172.0,http://www.imdb.com/title/tt98382,Star Trek V: The Final Frontier (1989),5.4,Action|Adventure|Fantasy,None
8,1446,Kolya (Kolja) (1996),116790.0,784.0,http://www.imdb.com/title/tt116790,Kolya (1996),7.8,Comedy|Drama|Music,None
13,423,Blown Away (1994),109303.0,178.0,http://www.imdb.com/title/tt109303,Blown Away (1994),6.2,Action|Crime|Drama,None
21,6625,Camp Nowhere (1994),109369.0,17745.0,http://www.imdb.com/title/tt109369,Camp Nowhere (1994),5.9,Adventure|Comedy|Family,None
...,...,...,...,...,...,...,...,...,...
2839,80162,"Horde, The (La Horde) (2009)",1183276.0,37905.0,http://www.imdb.com/title/tt1183276,La horde (2009),5.9,Action|Horror,None
2865,163653,David Cross: Making America Great Again (2016),5943936.0,408546.0,http://www.imdb.com/title/tt5943936,David Cross: Making America Great Again (2016),6.7,Comedy,None
2868,25833,Camille (1936),28683.0,43874.0,http://www.imdb.com/title/tt28683,Camille (1936),7.5,Drama|Romance,None
2871,2616,Dick Tracy (1990),99422.0,8592.0,http://www.imdb.com/title/tt99422,Dick Tracy (1990),6.1,Action|Comedy|Crime,None


In [919]:
i = 0 
for idx in df_test_faltantes.index:
    tmdb_id = df_test_faltantes.loc[idx, 'tmdbId']
    url = get_poster_by_id(tmdb_id, size='w185')
    df_test_faltantes.loc[idx, 'Poster'] = url
    i = i + 1
    print(f"Count: {i}, Índice: {idx}, Poster: {url}")

Count: 1, Índice: 0, Poster: https://image.tmdb.org/t/p/w185/jRzcb9O8WdGmJdz8EESEaBEuZZJ.jpg
Count: 2, Índice: 1, Poster: https://image.tmdb.org/t/p/w185/uiXr41VLYsuug3CZbFrKLSNahuZ.jpg
Count: 3, Índice: 8, Poster: https://image.tmdb.org/t/p/w185/fB0cGiEOngfsfyDHXIpFOmEwFgi.jpg
Count: 4, Índice: 13, Poster: https://image.tmdb.org/t/p/w185/pLmMaqMgJMOOtua41l7ocvzLS1t.jpg
Count: 5, Índice: 21, Poster: https://image.tmdb.org/t/p/w185/5DmempuEgWyHCu5lMVyICTRgZUp.jpg
Count: 6, Índice: 23, Poster: https://image.tmdb.org/t/p/w185/jlhe2SKWqW3F7BEQl3sOkgXLtkW.jpg
Count: 7, Índice: 27, Poster: https://image.tmdb.org/t/p/w185/qAECAgqjSFRVZzEEGbFAZqZNiDh.jpg
Count: 8, Índice: 28, Poster: https://image.tmdb.org/t/p/w185/9V1ASfnxMZxrkqIvVQYySE5qSwd.jpg
Count: 9, Índice: 54, Poster: https://image.tmdb.org/t/p/w185/tVyINAsNGSgD1OIstqwCcs7wyGH.jpg
Count: 10, Índice: 68, Poster: https://image.tmdb.org/t/p/w185/qDNOzfkgneIkJHQmP2Bk68LSDqR.jpg
Count: 11, Índice: 74, Poster: https://image.tmdb.org/t/p/w185

In [920]:
df_test_faltantes

,movieId,title,imdbId,tmdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,45635,"Notorious Bettie Page, The (2005)",404802.0,15402.0,http://www.imdb.com/title/tt404802,The Notorious Bettie Page (2005),6.6,Biography|Drama,https://image.tmdb.org/t/p/w185/jRzcb9O8WdGmJd...
1,1373,Star Trek V: The Final Frontier (1989),98382.0,172.0,http://www.imdb.com/title/tt98382,Star Trek V: The Final Frontier (1989),5.4,Action|Adventure|Fantasy,https://image.tmdb.org/t/p/w185/uiXr41VLYsuug3...
8,1446,Kolya (Kolja) (1996),116790.0,784.0,http://www.imdb.com/title/tt116790,Kolya (1996),7.8,Comedy|Drama|Music,https://image.tmdb.org/t/p/w185/fB0cGiEOngfsfy...
13,423,Blown Away (1994),109303.0,178.0,http://www.imdb.com/title/tt109303,Blown Away (1994),6.2,Action|Crime|Drama,https://image.tmdb.org/t/p/w185/pLmMaqMgJMOOtu...
21,6625,Camp Nowhere (1994),109369.0,17745.0,http://www.imdb.com/title/tt109369,Camp Nowhere (1994),5.9,Adventure|Comedy|Family,https://image.tmdb.org/t/p/w185/5DmempuEgWyHCu...
...,...,...,...,...,...,...,...,...,...
2839,80162,"Horde, The (La Horde) (2009)",1183276.0,37905.0,http://www.imdb.com/title/tt1183276,La horde (2009),5.9,Action|Horror,https://image.tmdb.org/t/p/w185/c3sYdfCg1vz6DG...
2865,163653,David Cross: Making America Great Again (2016),5943936.0,408546.0,http://www.imdb.com/title/tt5943936,David Cross: Making America Great Again (2016),6.7,Comedy,https://image.tmdb.org/t/p/w185/56PQtVg4SGhttb...
2868,25833,Camille (1936),28683.0,43874.0,http://www.imdb.com/title/tt28683,Camille (1936),7.5,Drama|Romance,https://image.tmdb.org/t/p/w185/78p9egiU16QpJN...
2871,2616,Dick Tracy (1990),99422.0,8592.0,http://www.imdb.com/title/tt99422,Dick Tracy (1990),6.1,Action|Comedy|Crime,https://image.tmdb.org/t/p/w185/AonBXZe8GwnW8f...


In [921]:
df_test_faltantes.isna().sum()

movieId       0
title         0
imdbId        0
tmdbId        0
Imdb Link     0
Title         0
IMDB Score    0
Genre         0
Poster        2
dtype: int64

Solo nos quedan 2 posters por buscar (los buscaremos mediante el nombre)

Ahora eliminaremos los features que no nos interesan:

In [922]:
df_test_faltantes_f = df_test_faltantes.drop(['Imdb Link', 'title', 'IMDB Score', 'Genre'], axis=1)
df_test_faltantes_f

,movieId,imdbId,tmdbId,Title,Poster
0,45635,404802.0,15402.0,The Notorious Bettie Page (2005),https://image.tmdb.org/t/p/w185/jRzcb9O8WdGmJd...
1,1373,98382.0,172.0,Star Trek V: The Final Frontier (1989),https://image.tmdb.org/t/p/w185/uiXr41VLYsuug3...
8,1446,116790.0,784.0,Kolya (1996),https://image.tmdb.org/t/p/w185/fB0cGiEOngfsfy...
13,423,109303.0,178.0,Blown Away (1994),https://image.tmdb.org/t/p/w185/pLmMaqMgJMOOtu...
21,6625,109369.0,17745.0,Camp Nowhere (1994),https://image.tmdb.org/t/p/w185/5DmempuEgWyHCu...
...,...,...,...,...,...
2839,80162,1183276.0,37905.0,La horde (2009),https://image.tmdb.org/t/p/w185/c3sYdfCg1vz6DG...
2865,163653,5943936.0,408546.0,David Cross: Making America Great Again (2016),https://image.tmdb.org/t/p/w185/56PQtVg4SGhttb...
2868,25833,28683.0,43874.0,Camille (1936),https://image.tmdb.org/t/p/w185/78p9egiU16QpJN...
2871,2616,99422.0,8592.0,Dick Tracy (1990),https://image.tmdb.org/t/p/w185/AonBXZe8GwnW8f...


In [923]:
df_test_faltantes_final = df_test_faltantes_f[df_test_faltantes_f.isna().any(axis=1)]
df_test_faltantes_final

,movieId,imdbId,tmdbId,Title,Poster
1228,77177,884762.0,141714.0,Wild China,None
1713,163809,3718778.0,409696.0,Over the Garden Wall,None


In [924]:
import re

In [925]:
for idx in df_test_faltantes_final.index:
    full_title = df_test_faltantes_final.at[idx, 'Title']
    m = re.match(r'^(.*)\s+\((\d{4})\)$', full_title)
    if m:
        title_cleaned = m.group(1)
        year = int(m.group(2))
    else:
        title_cleaned = full_title
        year = None
    if year is not None:
        url = get_poster_by_title(title_cleaned, year, size='w185')
    else:
        url = get_poster_by_title(title_cleaned, size='w185')
    df_test_faltantes_final.at[idx, 'Poster'] = url
    print(f"[{idx}] {title_cleaned!r} ({year}), url={url}")

[1228] 'Wild China' (None), url=https://image.tmdb.org/t/p/w185/9QjO5CbEDPfEwyTMUch2hWWC8tw.jpg
[1713] 'Over the Garden Wall' (None), url=https://image.tmdb.org/t/p/w185/uNd8daq7xpSo7GeXvAq04OY0xGr.jpg


In [926]:
df_test_faltantes_final

,movieId,imdbId,tmdbId,Title,Poster
1228,77177,884762.0,141714.0,Wild China,https://image.tmdb.org/t/p/w185/9QjO5CbEDPfEwy...
1713,163809,3718778.0,409696.0,Over the Garden Wall,https://image.tmdb.org/t/p/w185/uNd8daq7xpSo7G...


In [927]:
df_test_faltantes_f = df_test_faltantes_f.dropna()
df_test_faltantes_f.isna().sum()

movieId    0
imdbId     0
tmdbId     0
Title      0
Poster     0
dtype: int64

Concatenamos df_test_faltantes_final y df_test_faltantes_f (para esto eliminamos los 4 samples que no encontramos posters)

In [928]:
df_test_falt = pd.concat([df_test_faltantes_f, df_test_faltantes_final], axis=0, ignore_index=False)
df_test_falt

,movieId,imdbId,tmdbId,Title,Poster
0,45635,404802.0,15402.0,The Notorious Bettie Page (2005),https://image.tmdb.org/t/p/w185/jRzcb9O8WdGmJd...
1,1373,98382.0,172.0,Star Trek V: The Final Frontier (1989),https://image.tmdb.org/t/p/w185/uiXr41VLYsuug3...
8,1446,116790.0,784.0,Kolya (1996),https://image.tmdb.org/t/p/w185/fB0cGiEOngfsfy...
13,423,109303.0,178.0,Blown Away (1994),https://image.tmdb.org/t/p/w185/pLmMaqMgJMOOtu...
21,6625,109369.0,17745.0,Camp Nowhere (1994),https://image.tmdb.org/t/p/w185/5DmempuEgWyHCu...
...,...,...,...,...,...
2868,25833,28683.0,43874.0,Camille (1936),https://image.tmdb.org/t/p/w185/78p9egiU16QpJN...
2871,2616,99422.0,8592.0,Dick Tracy (1990),https://image.tmdb.org/t/p/w185/AonBXZe8GwnW8f...
2872,3895,204626.0,10685.0,The Watcher (2000),https://image.tmdb.org/t/p/w185/hsMvndtd3KPAvJ...
1228,77177,884762.0,141714.0,Wild China,https://image.tmdb.org/t/p/w185/9QjO5CbEDPfEwy...


In [929]:
df_test_falt = df_test_falt[['movieId', 'Poster']]
df_test_falt

,movieId,Poster
0,45635,https://image.tmdb.org/t/p/w185/jRzcb9O8WdGmJd...
1,1373,https://image.tmdb.org/t/p/w185/uiXr41VLYsuug3...
8,1446,https://image.tmdb.org/t/p/w185/fB0cGiEOngfsfy...
13,423,https://image.tmdb.org/t/p/w185/pLmMaqMgJMOOtu...
21,6625,https://image.tmdb.org/t/p/w185/5DmempuEgWyHCu...
...,...,...
2868,25833,https://image.tmdb.org/t/p/w185/78p9egiU16QpJN...
2871,2616,https://image.tmdb.org/t/p/w185/AonBXZe8GwnW8f...
2872,3895,https://image.tmdb.org/t/p/w185/hsMvndtd3KPAvJ...
1228,77177,https://image.tmdb.org/t/p/w185/9QjO5CbEDPfEwy...


In [930]:
df_test_falt = extract_features_from_dataframe(df_test_falt, id="movieId", url="Poster")
df_test_falt

Usando 7 hilos...


100%|██████████| 573/573 [00:25<00:00, 22.81it/s]


,movieId,0,1,2,3,4,5,6,7,8,...,103,104,105,106,107,108,109,110,111,112
0,45635,6162.0,2278.0,2924.0,10044.0,27722.0,44.0,1.0,0.0,0.0,...,0.082306,0.119755,0.125355,0.001386,3.229034e-07,1.186379e-11,1.634684e-11,2.271230e-22,9.198736e-15,-1.544055e-23
1,1373,8272.0,6013.0,1133.0,2019.0,3402.0,987.0,983.0,963.0,656.0,...,0.082481,0.104278,0.117383,0.002500,1.775627e-06,6.092452e-10,3.633303e-10,1.676652e-19,4.785587e-13,-3.330950e-20
2,423,8362.0,12762.0,6145.0,945.0,345.0,484.0,439.0,252.0,218.0,...,0.069745,0.166148,0.113825,0.001915,1.005423e-06,4.822129e-11,8.423998e-11,4.031287e-21,8.385387e-14,-3.546173e-21
3,694,13495.0,1877.0,16865.0,9631.0,3774.0,2112.0,967.0,171.0,163.0,...,0.057632,0.325919,0.105503,0.002144,6.375140e-07,8.601322e-10,3.360770e-10,-1.559807e-19,-2.614591e-13,9.121350e-20
4,1446,4516.0,7577.0,22697.0,2347.0,1342.0,1698.0,779.0,487.0,925.0,...,0.070251,0.126230,0.120027,0.001913,5.049203e-07,2.510712e-10,3.645898e-10,6.857043e-20,2.506204e-13,8.640515e-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,2616,3594.0,1280.0,1478.0,2098.0,1858.0,792.0,363.0,152.0,236.0,...,0.069084,0.462882,0.101186,0.004614,7.106587e-06,6.849555e-09,4.348748e-09,2.373058e-17,1.157179e-11,-4.233443e-19
569,26498,2099.0,2078.0,3665.0,2757.0,920.0,831.0,304.0,77.0,137.0,...,0.106397,0.120766,0.181703,0.001905,6.463039e-07,4.988540e-13,1.591549e-11,-6.942993e-24,1.134961e-14,4.430456e-23
570,3895,27813.0,2833.0,9166.0,5403.0,1417.0,538.0,80.0,26.0,52.0,...,0.034668,0.530352,0.067587,0.003940,1.456198e-07,1.176604e-09,1.031786e-09,1.055187e-18,-2.373732e-14,-4.230712e-19
571,77177,3423.0,15.0,7.0,312.0,155.0,656.0,90.0,66.0,172.0,...,0.087384,0.222254,0.160061,0.001081,1.762598e-07,6.327731e-12,7.105992e-12,4.701096e-23,2.973017e-15,7.776463e-24


Concatenamos ahora el df_test_falt y df_imgs_features

In [931]:
df_test_falt.index.duplicated().any()

np.False_

In [932]:
df_imgs_features.index.duplicated().any()

np.False_

In [933]:
df_imgs_features_c = pd.concat([df_imgs_features, df_test_falt], axis=0, ignore_index=True)
df_imgs_features_c

,movieId,0,1,2,3,4,5,6,7,8,...,103,104,105,106,107,108,109,110,111,112
0,141994,5196.0,2374.0,3968.0,3703.0,2795.0,2252.0,1144.0,524.0,578.0,...,0.102079,0.121474,0.195875,0.001201,1.406197e-07,2.476164e-11,1.321669e-11,2.380754e-22,4.909406e-15,-2.207549e-23
1,389,14316.0,12024.0,5204.0,1361.0,617.0,755.0,573.0,425.0,220.0,...,0.085247,0.134800,0.114667,0.002331,2.463068e-07,2.112036e-10,7.919602e-10,-5.472358e-20,3.915598e-13,-3.192397e-19
2,93040,27903.0,9873.0,635.0,122.0,42.0,162.0,15.0,9.0,56.0,...,0.056319,0.315914,0.110382,0.002082,4.497876e-07,6.595129e-10,4.500493e-10,2.411771e-19,2.831688e-13,4.417557e-20
3,7325,14449.0,4762.0,4265.0,2567.0,1030.0,455.0,156.0,57.0,97.0,...,0.075857,0.274459,0.143800,0.001395,2.607370e-07,1.777852e-11,2.348040e-13,-3.302922e-25,1.001093e-16,-3.479332e-25
4,8920,4337.0,920.0,518.0,790.0,11560.0,7018.0,615.0,113.0,95.0,...,0.066262,0.440237,0.093366,0.002354,8.109859e-07,3.010085e-12,2.227391e-11,1.633792e-22,2.004052e-14,8.106013e-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2616,3594.0,1280.0,1478.0,2098.0,1858.0,792.0,363.0,152.0,236.0,...,0.069084,0.462882,0.101186,0.004614,7.106587e-06,6.849555e-09,4.348748e-09,2.373058e-17,1.157179e-11,-4.233443e-19
2712,26498,2099.0,2078.0,3665.0,2757.0,920.0,831.0,304.0,77.0,137.0,...,0.106397,0.120766,0.181703,0.001905,6.463039e-07,4.988540e-13,1.591549e-11,-6.942993e-24,1.134961e-14,4.430456e-23
2713,3895,27813.0,2833.0,9166.0,5403.0,1417.0,538.0,80.0,26.0,52.0,...,0.034668,0.530352,0.067587,0.003940,1.456198e-07,1.176604e-09,1.031786e-09,1.055187e-18,-2.373732e-14,-4.230712e-19
2714,77177,3423.0,15.0,7.0,312.0,155.0,656.0,90.0,66.0,172.0,...,0.087384,0.222254,0.160061,0.001081,1.762598e-07,6.327731e-12,7.105992e-12,4.701096e-23,2.973017e-15,7.776463e-24


Hacemos una copia para usarla mas adelante

In [934]:
df_imgs_features_copy = df_imgs_features_c.copy()
df_imgs_features_copy

,movieId,0,1,2,3,4,5,6,7,8,...,103,104,105,106,107,108,109,110,111,112
0,141994,5196.0,2374.0,3968.0,3703.0,2795.0,2252.0,1144.0,524.0,578.0,...,0.102079,0.121474,0.195875,0.001201,1.406197e-07,2.476164e-11,1.321669e-11,2.380754e-22,4.909406e-15,-2.207549e-23
1,389,14316.0,12024.0,5204.0,1361.0,617.0,755.0,573.0,425.0,220.0,...,0.085247,0.134800,0.114667,0.002331,2.463068e-07,2.112036e-10,7.919602e-10,-5.472358e-20,3.915598e-13,-3.192397e-19
2,93040,27903.0,9873.0,635.0,122.0,42.0,162.0,15.0,9.0,56.0,...,0.056319,0.315914,0.110382,0.002082,4.497876e-07,6.595129e-10,4.500493e-10,2.411771e-19,2.831688e-13,4.417557e-20
3,7325,14449.0,4762.0,4265.0,2567.0,1030.0,455.0,156.0,57.0,97.0,...,0.075857,0.274459,0.143800,0.001395,2.607370e-07,1.777852e-11,2.348040e-13,-3.302922e-25,1.001093e-16,-3.479332e-25
4,8920,4337.0,920.0,518.0,790.0,11560.0,7018.0,615.0,113.0,95.0,...,0.066262,0.440237,0.093366,0.002354,8.109859e-07,3.010085e-12,2.227391e-11,1.633792e-22,2.004052e-14,8.106013e-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2616,3594.0,1280.0,1478.0,2098.0,1858.0,792.0,363.0,152.0,236.0,...,0.069084,0.462882,0.101186,0.004614,7.106587e-06,6.849555e-09,4.348748e-09,2.373058e-17,1.157179e-11,-4.233443e-19
2712,26498,2099.0,2078.0,3665.0,2757.0,920.0,831.0,304.0,77.0,137.0,...,0.106397,0.120766,0.181703,0.001905,6.463039e-07,4.988540e-13,1.591549e-11,-6.942993e-24,1.134961e-14,4.430456e-23
2713,3895,27813.0,2833.0,9166.0,5403.0,1417.0,538.0,80.0,26.0,52.0,...,0.034668,0.530352,0.067587,0.003940,1.456198e-07,1.176604e-09,1.031786e-09,1.055187e-18,-2.373732e-14,-4.230712e-19
2714,77177,3423.0,15.0,7.0,312.0,155.0,656.0,90.0,66.0,172.0,...,0.087384,0.222254,0.160061,0.001081,1.762598e-07,6.327731e-12,7.105992e-12,4.701096e-23,2.973017e-15,7.776463e-24


Ahora, hacemos el mismo tratamiento por el que pasamos el train

In [841]:
%pip install scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [935]:
from sklearn.feature_selection import VarianceThreshold

In [936]:
df_imgs_features_noId = df_imgs_features_c.drop('movieId', axis=1)
df_imgs_features_noId

,0,1,2,3,4,5,6,7,8,9,...,103,104,105,106,107,108,109,110,111,112
0,5196.0,2374.0,3968.0,3703.0,2795.0,2252.0,1144.0,524.0,578.0,346.0,...,0.102079,0.121474,0.195875,0.001201,1.406197e-07,2.476164e-11,1.321669e-11,2.380754e-22,4.909406e-15,-2.207549e-23
1,14316.0,12024.0,5204.0,1361.0,617.0,755.0,573.0,425.0,220.0,133.0,...,0.085247,0.134800,0.114667,0.002331,2.463068e-07,2.112036e-10,7.919602e-10,-5.472358e-20,3.915598e-13,-3.192397e-19
2,27903.0,9873.0,635.0,122.0,42.0,162.0,15.0,9.0,56.0,3.0,...,0.056319,0.315914,0.110382,0.002082,4.497876e-07,6.595129e-10,4.500493e-10,2.411771e-19,2.831688e-13,4.417557e-20
3,14449.0,4762.0,4265.0,2567.0,1030.0,455.0,156.0,57.0,97.0,58.0,...,0.075857,0.274459,0.143800,0.001395,2.607370e-07,1.777852e-11,2.348040e-13,-3.302922e-25,1.001093e-16,-3.479332e-25
4,4337.0,920.0,518.0,790.0,11560.0,7018.0,615.0,113.0,95.0,42.0,...,0.066262,0.440237,0.093366,0.002354,8.109859e-07,3.010085e-12,2.227391e-11,1.633792e-22,2.004052e-14,8.106013e-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,3594.0,1280.0,1478.0,2098.0,1858.0,792.0,363.0,152.0,236.0,54.0,...,0.069084,0.462882,0.101186,0.004614,7.106587e-06,6.849555e-09,4.348748e-09,2.373058e-17,1.157179e-11,-4.233443e-19
2712,2099.0,2078.0,3665.0,2757.0,920.0,831.0,304.0,77.0,137.0,2.0,...,0.106397,0.120766,0.181703,0.001905,6.463039e-07,4.988540e-13,1.591549e-11,-6.942993e-24,1.134961e-14,4.430456e-23
2713,27813.0,2833.0,9166.0,5403.0,1417.0,538.0,80.0,26.0,52.0,3.0,...,0.034668,0.530352,0.067587,0.003940,1.456198e-07,1.176604e-09,1.031786e-09,1.055187e-18,-2.373732e-14,-4.230712e-19
2714,3423.0,15.0,7.0,312.0,155.0,656.0,90.0,66.0,172.0,0.0,...,0.087384,0.222254,0.160061,0.001081,1.762598e-07,6.327731e-12,7.105992e-12,4.701096e-23,2.973017e-15,7.776463e-24


In [937]:
selector = VarianceThreshold(threshold=0.01) # El umbral elegido sera 0.01
df_imgs_features_noId_filt_1 = selector.fit_transform(df_imgs_features_noId)
df_imgs_features_noId_filt_1 = pd.DataFrame(df_imgs_features_noId_filt_1)
display(df_imgs_features_noId_filt_1)

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,5196.0,2374.0,3968.0,3703.0,2795.0,2252.0,1144.0,524.0,578.0,346.0,...,2066.0,2071.0,1901.0,1815.0,1646.0,1566.0,1437.0,1416.0,5292.0,0.121474
1,14316.0,12024.0,5204.0,1361.0,617.0,755.0,573.0,425.0,220.0,133.0,...,842.0,547.0,440.0,298.0,265.0,282.0,253.0,228.0,457.0,0.134800
2,27903.0,9873.0,635.0,122.0,42.0,162.0,15.0,9.0,56.0,3.0,...,1959.0,1653.0,1511.0,1501.0,1462.0,1140.0,1019.0,1029.0,1977.0,0.315914
3,14449.0,4762.0,4265.0,2567.0,1030.0,455.0,156.0,57.0,97.0,58.0,...,941.0,1022.0,1126.0,1289.0,1594.0,1782.0,3579.0,7603.0,8155.0,0.274459
4,4337.0,920.0,518.0,790.0,11560.0,7018.0,615.0,113.0,95.0,42.0,...,189.0,142.0,136.0,179.0,186.0,239.0,347.0,613.0,17351.0,0.440237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,3594.0,1280.0,1478.0,2098.0,1858.0,792.0,363.0,152.0,236.0,54.0,...,659.0,213.0,190.0,216.0,242.0,254.0,250.0,250.0,842.0,0.462882
2712,2099.0,2078.0,3665.0,2757.0,920.0,831.0,304.0,77.0,137.0,2.0,...,2611.0,2161.0,1156.0,442.0,428.0,534.0,650.0,1381.0,2488.0,0.120766
2713,27813.0,2833.0,9166.0,5403.0,1417.0,538.0,80.0,26.0,52.0,3.0,...,421.0,446.0,464.0,424.0,519.0,507.0,443.0,383.0,594.0,0.530352
2714,3423.0,15.0,7.0,312.0,155.0,656.0,90.0,66.0,172.0,0.0,...,5210.0,2890.0,1796.0,2050.0,2021.0,4185.0,7632.0,1299.0,8699.0,0.222254


In [938]:
corr_matrix = df_imgs_features_noId_filt_1.corr().abs() #df_imgs_features_noId_filt_1

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

umbral = 0.9
columnas_a_eliminar = [col for col in upper.columns if any(upper[col] > umbral)]

df_imgs_features_noId_filt_2 = df_imgs_features_noId_filt_1.drop(columns=columnas_a_eliminar)

display(df_imgs_features_noId_filt_2)

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,5196.0,2374.0,3968.0,3703.0,2795.0,2252.0,1144.0,524.0,578.0,346.0,...,2066.0,2071.0,1901.0,1815.0,1646.0,1566.0,1437.0,1416.0,5292.0,0.121474
1,14316.0,12024.0,5204.0,1361.0,617.0,755.0,573.0,425.0,220.0,133.0,...,842.0,547.0,440.0,298.0,265.0,282.0,253.0,228.0,457.0,0.134800
2,27903.0,9873.0,635.0,122.0,42.0,162.0,15.0,9.0,56.0,3.0,...,1959.0,1653.0,1511.0,1501.0,1462.0,1140.0,1019.0,1029.0,1977.0,0.315914
3,14449.0,4762.0,4265.0,2567.0,1030.0,455.0,156.0,57.0,97.0,58.0,...,941.0,1022.0,1126.0,1289.0,1594.0,1782.0,3579.0,7603.0,8155.0,0.274459
4,4337.0,920.0,518.0,790.0,11560.0,7018.0,615.0,113.0,95.0,42.0,...,189.0,142.0,136.0,179.0,186.0,239.0,347.0,613.0,17351.0,0.440237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,3594.0,1280.0,1478.0,2098.0,1858.0,792.0,363.0,152.0,236.0,54.0,...,659.0,213.0,190.0,216.0,242.0,254.0,250.0,250.0,842.0,0.462882
2712,2099.0,2078.0,3665.0,2757.0,920.0,831.0,304.0,77.0,137.0,2.0,...,2611.0,2161.0,1156.0,442.0,428.0,534.0,650.0,1381.0,2488.0,0.120766
2713,27813.0,2833.0,9166.0,5403.0,1417.0,538.0,80.0,26.0,52.0,3.0,...,421.0,446.0,464.0,424.0,519.0,507.0,443.0,383.0,594.0,0.530352
2714,3423.0,15.0,7.0,312.0,155.0,656.0,90.0,66.0,172.0,0.0,...,5210.0,2890.0,1796.0,2050.0,2021.0,4185.0,7632.0,1299.0,8699.0,0.222254


In [939]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [940]:
scaler = StandardScaler()
# df_imgs_features_noId_scl = scaler.fit_transform(df_imgs_features_noId_filt_2)
# df_imgs_features_noId_scl = pd.DataFrame(df_imgs_features_noId_scl)
df_imgs_features_noId_scl = pd.DataFrame(scaler.fit_transform(df_imgs_features_noId_filt_2), index=df_imgs_features_noId_filt_2.index)
df_imgs_features_noId_scl

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,-0.459303,-0.416011,-0.159977,-0.016186,0.064837,0.113424,0.080828,0.068878,0.090620,0.088453,...,0.727370,0.590614,0.520900,0.243122,0.067442,-0.066214,-0.201789,-0.272786,-0.218133,-0.798747
1,0.504416,1.747607,0.082677,-0.510707,-0.447126,-0.388996,-0.175008,-0.017373,-0.218084,-0.182446,...,-0.286801,-0.526347,-0.761067,-0.716394,-0.746834,-0.672793,-0.646996,-0.678559,-0.779678,-0.692113
2,1.940166,1.265333,-0.814320,-0.772326,-0.582286,-0.588017,-0.425019,-0.379799,-0.359501,-0.347784,...,0.638713,0.284256,0.178691,0.044515,-0.041050,-0.267462,-0.358965,-0.404970,-0.603142,0.757129
3,0.518470,0.119400,-0.101670,-0.256056,-0.350046,-0.489681,-0.361844,-0.337981,-0.324147,-0.277833,...,-0.204773,-0.178213,-0.159131,-0.089577,0.036781,0.035827,0.603644,1.840440,0.114381,0.425414
4,-0.550074,-0.742012,-0.837290,-0.631276,2.125146,1.712978,-0.156190,-0.289193,-0.325871,-0.298182,...,-0.827858,-0.823176,-1.027815,-0.791662,-0.793415,-0.693107,-0.611650,-0.547058,1.182421,1.751945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,-0.628587,-0.661296,-0.648820,-0.355087,-0.155415,-0.376578,-0.269098,-0.255215,-0.204287,-0.282920,...,-0.438430,-0.771140,-0.980432,-0.768259,-0.760396,-0.686020,-0.648124,-0.671044,-0.734963,1.933143
2712,-0.786565,-0.482377,-0.219463,-0.215937,-0.375902,-0.363489,-0.295533,-0.320556,-0.289655,-0.349055,...,1.178941,0.656576,-0.132807,-0.625312,-0.650725,-0.553745,-0.497716,-0.284741,-0.543794,-0.804409
2713,1.930655,-0.313099,0.860507,0.342775,-0.259077,-0.461825,-0.395896,-0.364988,-0.362950,-0.347784,...,-0.635630,-0.600371,-0.740008,-0.636697,-0.597069,-0.566500,-0.575552,-0.625617,-0.763766,2.473030
2714,-0.646657,-0.944921,-0.937611,-0.732207,-0.555724,-0.422222,-0.391416,-0.330140,-0.259474,-0.351599,...,3.332398,1.190870,0.428767,0.391762,0.288552,1.171036,2.127649,-0.312749,0.177562,0.007680


In [941]:
df_imgs_features_c.index.duplicated().any()

np.False_

In [942]:
df_imgs_features_noId_scl.index.duplicated().any()

np.False_

In [943]:
df_imgs_features_noId_scl["movieId"] = df_imgs_features_c["movieId"]

cols = ['movieId'] + [col for col in df_imgs_features_noId_scl.columns if col != 'movieId']
df_imgs_features_scl = df_imgs_features_noId_scl[cols]

display(df_imgs_features_scl)

,movieId,0,1,2,3,4,5,6,7,8,...,87,88,89,90,91,92,93,94,95,96
0,141994,-0.459303,-0.416011,-0.159977,-0.016186,0.064837,0.113424,0.080828,0.068878,0.090620,...,0.727370,0.590614,0.520900,0.243122,0.067442,-0.066214,-0.201789,-0.272786,-0.218133,-0.798747
1,389,0.504416,1.747607,0.082677,-0.510707,-0.447126,-0.388996,-0.175008,-0.017373,-0.218084,...,-0.286801,-0.526347,-0.761067,-0.716394,-0.746834,-0.672793,-0.646996,-0.678559,-0.779678,-0.692113
2,93040,1.940166,1.265333,-0.814320,-0.772326,-0.582286,-0.588017,-0.425019,-0.379799,-0.359501,...,0.638713,0.284256,0.178691,0.044515,-0.041050,-0.267462,-0.358965,-0.404970,-0.603142,0.757129
3,7325,0.518470,0.119400,-0.101670,-0.256056,-0.350046,-0.489681,-0.361844,-0.337981,-0.324147,...,-0.204773,-0.178213,-0.159131,-0.089577,0.036781,0.035827,0.603644,1.840440,0.114381,0.425414
4,8920,-0.550074,-0.742012,-0.837290,-0.631276,2.125146,1.712978,-0.156190,-0.289193,-0.325871,...,-0.827858,-0.823176,-1.027815,-0.791662,-0.793415,-0.693107,-0.611650,-0.547058,1.182421,1.751945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2616,-0.628587,-0.661296,-0.648820,-0.355087,-0.155415,-0.376578,-0.269098,-0.255215,-0.204287,...,-0.438430,-0.771140,-0.980432,-0.768259,-0.760396,-0.686020,-0.648124,-0.671044,-0.734963,1.933143
2712,26498,-0.786565,-0.482377,-0.219463,-0.215937,-0.375902,-0.363489,-0.295533,-0.320556,-0.289655,...,1.178941,0.656576,-0.132807,-0.625312,-0.650725,-0.553745,-0.497716,-0.284741,-0.543794,-0.804409
2713,3895,1.930655,-0.313099,0.860507,0.342775,-0.259077,-0.461825,-0.395896,-0.364988,-0.362950,...,-0.635630,-0.600371,-0.740008,-0.636697,-0.597069,-0.566500,-0.575552,-0.625617,-0.763766,2.473030
2714,77177,-0.646657,-0.944921,-0.937611,-0.732207,-0.555724,-0.422222,-0.391416,-0.330140,-0.259474,...,3.332398,1.190870,0.428767,0.391762,0.288552,1.171036,2.127649,-0.312749,0.177562,0.007680


In [944]:
df_test_f = pd.merge(df_imgs_features_scl, df_genere_extractor_d, how='inner', on='movieId')
df_test_f

,movieId,0,1,2,3,4,5,6,7,8,...,Mystery,News,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,141994,-0.459303,-0.416011,-0.159977,-0.016186,0.064837,0.113424,0.080828,0.068878,0.090620,...,0,0,0,0,0,0,0,0,0,0
1,389,0.504416,1.747607,0.082677,-0.510707,-0.447126,-0.388996,-0.175008,-0.017373,-0.218084,...,0,0,1,0,0,0,0,0,0,0
2,93040,1.940166,1.265333,-0.814320,-0.772326,-0.582286,-0.588017,-0.425019,-0.379799,-0.359501,...,0,0,0,0,0,0,0,0,1,0
3,7325,0.518470,0.119400,-0.101670,-0.256056,-0.350046,-0.489681,-0.361844,-0.337981,-0.324147,...,0,0,0,0,0,0,0,0,0,0
4,8920,-0.550074,-0.742012,-0.837290,-0.631276,2.125146,1.712978,-0.156190,-0.289193,-0.325871,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2616,-0.628587,-0.661296,-0.648820,-0.355087,-0.155415,-0.376578,-0.269098,-0.255215,-0.204287,...,0,0,0,0,0,0,0,0,0,0
2712,26498,-0.786565,-0.482377,-0.219463,-0.215937,-0.375902,-0.363489,-0.295533,-0.320556,-0.289655,...,0,0,1,0,0,0,0,0,0,0
2713,3895,1.930655,-0.313099,0.860507,0.342775,-0.259077,-0.461825,-0.395896,-0.364988,-0.362950,...,1,0,0,0,0,0,0,0,0,0
2714,77177,-0.646657,-0.944921,-0.937611,-0.732207,-0.555724,-0.422222,-0.391416,-0.330140,-0.259474,...,0,0,0,0,0,0,0,0,0,0


In [945]:
df_test_f

,movieId,0,1,2,3,4,5,6,7,8,...,Mystery,News,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,141994,-0.459303,-0.416011,-0.159977,-0.016186,0.064837,0.113424,0.080828,0.068878,0.090620,...,0,0,0,0,0,0,0,0,0,0
1,389,0.504416,1.747607,0.082677,-0.510707,-0.447126,-0.388996,-0.175008,-0.017373,-0.218084,...,0,0,1,0,0,0,0,0,0,0
2,93040,1.940166,1.265333,-0.814320,-0.772326,-0.582286,-0.588017,-0.425019,-0.379799,-0.359501,...,0,0,0,0,0,0,0,0,1,0
3,7325,0.518470,0.119400,-0.101670,-0.256056,-0.350046,-0.489681,-0.361844,-0.337981,-0.324147,...,0,0,0,0,0,0,0,0,0,0
4,8920,-0.550074,-0.742012,-0.837290,-0.631276,2.125146,1.712978,-0.156190,-0.289193,-0.325871,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2616,-0.628587,-0.661296,-0.648820,-0.355087,-0.155415,-0.376578,-0.269098,-0.255215,-0.204287,...,0,0,0,0,0,0,0,0,0,0
2712,26498,-0.786565,-0.482377,-0.219463,-0.215937,-0.375902,-0.363489,-0.295533,-0.320556,-0.289655,...,0,0,1,0,0,0,0,0,0,0
2713,3895,1.930655,-0.313099,0.860507,0.342775,-0.259077,-0.461825,-0.395896,-0.364988,-0.362950,...,1,0,0,0,0,0,0,0,0,0
2714,77177,-0.646657,-0.944921,-0.937611,-0.732207,-0.555724,-0.422222,-0.391416,-0.330140,-0.259474,...,0,0,0,0,0,0,0,0,0,0


In [946]:
df_imgs_features_scl.to_csv('df_test.csv', index=False)

In [947]:
df_test_f.to_csv('df_test_genere.csv', index=False)

Ahora, haremos una ampliacion del df_test.csv para abarcar la mayor cantidad de samples del test posible.

En este caso usaremos este dataframe df_imgs_features_copy, ya que este no a resevido el tratamiento, solo tiene todos los features visuales como tal:

In [948]:
df_imgs_features_copy

,movieId,0,1,2,3,4,5,6,7,8,...,103,104,105,106,107,108,109,110,111,112
0,141994,5196.0,2374.0,3968.0,3703.0,2795.0,2252.0,1144.0,524.0,578.0,...,0.102079,0.121474,0.195875,0.001201,1.406197e-07,2.476164e-11,1.321669e-11,2.380754e-22,4.909406e-15,-2.207549e-23
1,389,14316.0,12024.0,5204.0,1361.0,617.0,755.0,573.0,425.0,220.0,...,0.085247,0.134800,0.114667,0.002331,2.463068e-07,2.112036e-10,7.919602e-10,-5.472358e-20,3.915598e-13,-3.192397e-19
2,93040,27903.0,9873.0,635.0,122.0,42.0,162.0,15.0,9.0,56.0,...,0.056319,0.315914,0.110382,0.002082,4.497876e-07,6.595129e-10,4.500493e-10,2.411771e-19,2.831688e-13,4.417557e-20
3,7325,14449.0,4762.0,4265.0,2567.0,1030.0,455.0,156.0,57.0,97.0,...,0.075857,0.274459,0.143800,0.001395,2.607370e-07,1.777852e-11,2.348040e-13,-3.302922e-25,1.001093e-16,-3.479332e-25
4,8920,4337.0,920.0,518.0,790.0,11560.0,7018.0,615.0,113.0,95.0,...,0.066262,0.440237,0.093366,0.002354,8.109859e-07,3.010085e-12,2.227391e-11,1.633792e-22,2.004052e-14,8.106013e-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2616,3594.0,1280.0,1478.0,2098.0,1858.0,792.0,363.0,152.0,236.0,...,0.069084,0.462882,0.101186,0.004614,7.106587e-06,6.849555e-09,4.348748e-09,2.373058e-17,1.157179e-11,-4.233443e-19
2712,26498,2099.0,2078.0,3665.0,2757.0,920.0,831.0,304.0,77.0,137.0,...,0.106397,0.120766,0.181703,0.001905,6.463039e-07,4.988540e-13,1.591549e-11,-6.942993e-24,1.134961e-14,4.430456e-23
2713,3895,27813.0,2833.0,9166.0,5403.0,1417.0,538.0,80.0,26.0,52.0,...,0.034668,0.530352,0.067587,0.003940,1.456198e-07,1.176604e-09,1.031786e-09,1.055187e-18,-2.373732e-14,-4.230712e-19
2714,77177,3423.0,15.0,7.0,312.0,155.0,656.0,90.0,66.0,172.0,...,0.087384,0.222254,0.160061,0.001081,1.762598e-07,6.327731e-12,7.105992e-12,4.701096e-23,2.973017e-15,7.776463e-24


In [949]:
movieIds_faltantes = df_test[~df_test['movieId'].isin(df_imgs_features_copy['movieId'])]
movieIds_faltantes

,movieId,title
6,120807,John Mulaney: New In Town (2012)
16,176389,The Nut Job 2: Nutty by Nature (2017)
19,174479,Unedited Footage of a Bear (2014)
20,185029,A Quiet Place (2018)
67,177593,"Three Billboards Outside Ebbing, Missouri (2017)"
...,...,...
2900,875,Nothing to Lose (1994)
2907,180297,The Disaster Artist (2017)
2911,169912,Denis Leary: No Cure for Cancer (1993)
2912,165639,While You Were Fighting: A Thor Mockumentary (...


Ahora buscamos todas las peliculas que faltan por el titulo en TMDb

In [950]:
movieIds_faltantes['Poster'] = None
movieIds_faltantes

/var/folders/2w/09s3w8g13ljfzp1b6hpsfjym0000gn/T/ipykernel_1885/270378773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movieIds_faltantes['Poster'] = None


,movieId,title,Poster
6,120807,John Mulaney: New In Town (2012),None
16,176389,The Nut Job 2: Nutty by Nature (2017),None
19,174479,Unedited Footage of a Bear (2014),None
20,185029,A Quiet Place (2018),None
67,177593,"Three Billboards Outside Ebbing, Missouri (2017)",None
...,...,...,...
2900,875,Nothing to Lose (1994),None
2907,180297,The Disaster Artist (2017),None
2911,169912,Denis Leary: No Cure for Cancer (1993),None
2912,165639,While You Were Fighting: A Thor Mockumentary (...,None


In [951]:
for idx in movieIds_faltantes.index:
    full_title = movieIds_faltantes.at[idx, 'title']
    m = re.match(r'^(.*)\s+\((\d{4})\)$', full_title)
    if m:
        title_cleaned = m.group(1)
        year = int(m.group(2))
    else:
        title_cleaned = full_title
        year = None
    if year is not None:
        url = get_poster_by_title(title_cleaned, year, size='w185')
    else:
        url = get_poster_by_title(title_cleaned, size='w185')
    movieIds_faltantes.at[idx, 'Poster'] = url
    print(f"[{idx}] {title_cleaned!r} ({year}), url={url}")

[6] 'John Mulaney: New In Town' (2012), url=https://image.tmdb.org/t/p/w185/ufE8KAkLQT9u91AgDSc5Oxa3DjW.jpg
[16] 'The Nut Job 2: Nutty by Nature' (2017), url=https://image.tmdb.org/t/p/w185/xOfdQHNF9TlrdujyAjiKfUhxSXy.jpg
[19] 'Unedited Footage of a Bear' (2014), url=https://image.tmdb.org/t/p/w185/eBE8V51zojucBMzTN3DdN9H4Ct9.jpg
[20] 'A Quiet Place' (2018), url=https://image.tmdb.org/t/p/w185/nAU74GmpUk7t5iklEp3bufwDq4n.jpg
[67] 'Three Billboards Outside Ebbing, Missouri' (2017), url=https://image.tmdb.org/t/p/w185/uGMM9ZObmPUFrGqcbFMVyv8L1lU.jpg
[98] 'Rampage' (2018), url=https://image.tmdb.org/t/p/w185/MGADip4thVSErP34FAAfzFBTZ5.jpg
[122] "A Merry Friggin' Christmas" (2014), url=https://image.tmdb.org/t/p/w185/eR9muFXkNCknGdfTQSeDS6tdPdZ.jpg
[140] 'Violet & Daisy' (2011), url=https://image.tmdb.org/t/p/w185/aQawgKvHovftR0Ues1tlYrxLEWh.jpg
[145] 'Too Funny to Fail: The Life and Death of The Dana Carvey Show' (2017), url=None
[165] 'Tokyo Idols' (2017), url=https://image.tmdb.org/t/p/

In [952]:
movieIds_faltantes

,movieId,title,Poster
6,120807,John Mulaney: New In Town (2012),https://image.tmdb.org/t/p/w185/ufE8KAkLQT9u91...
16,176389,The Nut Job 2: Nutty by Nature (2017),https://image.tmdb.org/t/p/w185/xOfdQHNF9Tlrdu...
19,174479,Unedited Footage of a Bear (2014),https://image.tmdb.org/t/p/w185/eBE8V51zojucBM...
20,185029,A Quiet Place (2018),https://image.tmdb.org/t/p/w185/nAU74GmpUk7t5i...
67,177593,"Three Billboards Outside Ebbing, Missouri (2017)",https://image.tmdb.org/t/p/w185/uGMM9ZObmPUFrG...
...,...,...,...
2900,875,Nothing to Lose (1994),https://image.tmdb.org/t/p/w185/obRFIwQWBieexk...
2907,180297,The Disaster Artist (2017),https://image.tmdb.org/t/p/w185/2HuLGiyH0TPYxn...
2911,169912,Denis Leary: No Cure for Cancer (1993),https://image.tmdb.org/t/p/w185/bjybZeS8uPK0ne...
2912,165639,While You Were Fighting: A Thor Mockumentary (...,https://image.tmdb.org/t/p/w185/jVSmX89BvsQV2z...


In [953]:
movieIds_faltantes.isna().sum()

movieId     0
title       0
Poster     15
dtype: int64

In [954]:
movieIds_faltantes_nans_posters = movieIds_faltantes[movieIds_faltantes['Poster'].isnull()]
movieIds_faltantes_nans_posters

,movieId,title,Poster
145,181413,Too Funny to Fail: The Life and Death of The D...,None
313,26587,"Decalogue, The (Dekalog) (1989)",None
1352,95004,Superman/Doomsday (2007),None
1439,108192,Hotel Chevalier (Part 1 of 'The Darjeeling Lim...,None
1538,74282,Anne of Green Gables: The Sequel (a.k.a. Anne ...,None
1964,185135,Sherlock - A Study in Pink (2010),None
1966,7762,"Tinker, Tailor, Soldier, Spy (1979)",None
1972,137517,Carnival Magic (1981),None
2118,179135,Blue Planet II (2017),None
2188,96004,Dragon Ball Z: The History of Trunks (Doragon ...,None


In [955]:
x = list(movieIds_faltantes_nans_posters['movieId'])
x

[181413,
 26587,
 95004,
 108192,
 74282,
 185135,
 7762,
 137517,
 179135,
 96004,
 26510,
 171917,
 95499,
 106648,
 71160]

In [876]:
movieIds_faltantes = movieIds_faltantes.dropna()
movieIds_faltantes.isna().sum()

movieId    0
title      0
Poster     0
dtype: int64

Hacemos el mismo tratamiento que a los demas

In [877]:
movieIds_faltantes_ex = extract_features_from_dataframe(movieIds_faltantes, id="movieId", url="Poster")

df_test_complete = pd.concat([df_imgs_features_copy, movieIds_faltantes_ex], axis=0, ignore_index=True)

df_test_complete_noId = df_test_complete.drop('movieId', axis=1)

selector = VarianceThreshold(threshold=0.01) # El umbral elegido sera 0.01
df_test_complete_noId_filt_1 = selector.fit_transform(df_test_complete_noId)
df_test_complete_noId_filt_1 = pd.DataFrame(df_test_complete_noId_filt_1)

corr_matrix = df_test_complete_noId_filt_1.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

umbral = 0.9
columnas_a_eliminar = [col for col in upper.columns if any(upper[col] > umbral)]

df_test_complete_noId_filt_2 = df_test_complete_noId_filt_1.drop(columns=columnas_a_eliminar)

scaler = StandardScaler()
df_test_complete_noId_scl = pd.DataFrame(scaler.fit_transform(df_test_complete_noId_filt_2), index=df_test_complete_noId_filt_2.index)
df_test_complete_noId_scl

Usando 7 hilos...


100%|██████████| 192/192 [00:08<00:00, 23.07it/s]


,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,0.517531,1.737029,0.085255,-0.510213,-0.448220,-0.383794,-0.177703,-0.021340,-0.224063,-0.159248,...,-0.297545,-0.536926,-0.736687,-0.656957,-0.753945,-0.680455,-0.647540,-0.672423,-0.770060,-0.673726
1,-0.543296,-0.744337,-0.841299,-0.631803,2.178432,1.757406,-0.158780,-0.296532,-0.333039,-0.244090,...,-0.842633,-0.835054,-0.990425,-0.724374,-0.800999,-0.701198,-0.611932,-0.541990,1.209434,1.776116
2,1.961910,1.256354,-0.818165,-0.774048,-0.586238,-0.586529,-0.429111,-0.388262,-0.367040,-0.280451,...,0.634864,0.277221,0.157240,0.024577,-0.040993,-0.266571,-0.357372,-0.401054,-0.591960,0.778946
3,-0.451979,-0.419418,-0.159137,-0.011501,0.074566,0.128002,0.079561,0.065980,0.088044,0.039338,...,0.724181,0.584919,0.482760,0.202468,0.068601,-0.061075,-0.199029,-0.269943,-0.203536,-0.780613
4,0.531670,0.114219,-0.100412,-0.253404,-0.349088,-0.486358,-0.365583,-0.345925,-0.331296,-0.229173,...,-0.214905,-0.187269,-0.164106,-0.095527,0.037628,0.043120,0.612381,1.826139,0.131925,0.446446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2903,-0.976599,-0.944786,-0.929486,-0.774474,-0.568476,-0.518495,-0.418748,-0.389144,-0.344373,-0.268331,...,-0.626434,-0.687830,-0.906124,-0.694348,-0.800403,-0.740271,-0.682390,-0.684958,-0.181860,-0.662616
2904,-0.697227,-0.585229,-0.725628,-0.695047,-0.556474,-0.616956,-0.425056,-0.389144,-0.396681,-0.275789,...,-0.254973,-0.291062,-0.433703,-0.262653,-0.265540,-0.491361,-0.486167,-0.585693,-0.404134,-1.077613
2905,-0.999668,-0.781209,-0.361018,-0.451866,0.215704,-0.128751,0.025044,0.138306,0.097634,0.256571,...,0.594796,0.396472,0.118845,-0.232060,-0.372751,-0.377519,-0.336158,-0.299756,-0.540522,-0.959910
2906,0.146098,0.919144,0.267363,-0.070912,-0.205310,-0.399862,-0.311968,-0.275363,-0.144729,-0.220782,...,-0.054635,0.171220,0.107995,-0.072866,-0.269709,-0.293102,-0.184256,-0.066331,-0.163698,-0.505725


Ahora tenemos posters de peliculas menos 15 del total, lo cual es un porcentaje 0.5% de data faltante, lo cual es tan poco que podemos omitirlo

In [878]:
df_test_complete_noId_scl["movieId"] = df_test_complete["movieId"]

cols = ['movieId'] + [col for col in df_test_complete_noId_scl.columns if col != 'movieId']
df_test_complete_f = df_test_complete_noId_scl[cols]

display(df_test_complete_f)

,movieId,0,1,2,3,4,5,6,7,8,...,87,88,89,90,91,92,93,94,95,96
0,389,0.517531,1.737029,0.085255,-0.510213,-0.448220,-0.383794,-0.177703,-0.021340,-0.224063,...,-0.297545,-0.536926,-0.736687,-0.656957,-0.753945,-0.680455,-0.647540,-0.672423,-0.770060,-0.673726
1,8920,-0.543296,-0.744337,-0.841299,-0.631803,2.178432,1.757406,-0.158780,-0.296532,-0.333039,...,-0.842633,-0.835054,-0.990425,-0.724374,-0.800999,-0.701198,-0.611932,-0.541990,1.209434,1.776116
2,93040,1.961910,1.256354,-0.818165,-0.774048,-0.586238,-0.586529,-0.429111,-0.388262,-0.367040,...,0.634864,0.277221,0.157240,0.024577,-0.040993,-0.266571,-0.357372,-0.401054,-0.591960,0.778946
3,141994,-0.451979,-0.419418,-0.159137,-0.011501,0.074566,0.128002,0.079561,0.065980,0.088044,...,0.724181,0.584919,0.482760,0.202468,0.068601,-0.061075,-0.199029,-0.269943,-0.203536,-0.780613
4,7325,0.531670,0.114219,-0.100412,-0.253404,-0.349088,-0.486358,-0.365583,-0.345925,-0.331296,...,-0.214905,-0.187269,-0.164106,-0.095527,0.037628,0.043120,0.612381,1.826139,0.131925,0.446446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2903,180297,-0.976599,-0.944786,-0.929486,-0.774474,-0.568476,-0.518495,-0.418748,-0.389144,-0.344373,...,-0.626434,-0.687830,-0.906124,-0.694348,-0.800403,-0.740271,-0.682390,-0.684958,-0.181860,-0.662616
2904,165639,-0.697227,-0.585229,-0.725628,-0.695047,-0.556474,-0.616956,-0.425056,-0.389144,-0.396681,...,-0.254973,-0.291062,-0.433703,-0.262653,-0.265540,-0.491361,-0.486167,-0.585693,-0.404134,-1.077613
2905,27074,-0.999668,-0.781209,-0.361018,-0.451866,0.215704,-0.128751,0.025044,0.138306,0.097634,...,0.594796,0.396472,0.118845,-0.232060,-0.372751,-0.377519,-0.336158,-0.299756,-0.540522,-0.959910
2906,95149,0.146098,0.919144,0.267363,-0.070912,-0.205310,-0.399862,-0.311968,-0.275363,-0.144729,...,-0.054635,0.171220,0.107995,-0.072866,-0.269709,-0.293102,-0.184256,-0.066331,-0.163698,-0.505725


Ahora solo hace falta pasarlo por PCA y UMAP para usarlo en el clustering

# Reduccion de la dimencionalidad

In [760]:
%pip install umap-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [765]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap.umap_ as umap

In [766]:
def pca_umap_reduction(features, n_components_pca=50, n_components_umap=10, random_state=42):
    pca = PCA(n_components=n_components_pca, svd_solver='randomized', random_state=random_state)

    features_pca = pca.fit_transform(features)

    umap_reducer = umap.UMAP(
        n_components=n_components_umap,
        random_state=random_state,
        n_neighbors=15,
        min_dist=0.1,
        metric='euclidean'
    )
    
    features_final = umap_reducer.fit_transform(features_pca)

    return features_final

Tomamos los datasets generados anteriormente para usarlos en esta face

In [767]:
df_test_preReduced = pd.read_csv('../data/test/test_processing/df_test.csv')
df_test_genere_preReduced = pd.read_csv('../data/test/test_processing/df_test_genere.csv')
display(df_test_preReduced)
display(df_test_genere_preReduced)

,movieId,0,1,2,3,4,5,6,7,8,...,87,88,89,90,91,92,93,94,95,96
0,8920,-0.550110,-0.742012,-0.837285,-0.631286,2.125147,1.712984,-0.156185,-0.289170,-0.325896,...,-0.827856,-0.823194,-1.027804,-0.791680,-0.793423,-0.693096,-0.611653,-0.547066,1.182421,1.751948
1,93040,1.940170,1.265333,-0.814315,-0.772337,-0.582296,-0.588000,-0.425014,-0.379776,-0.359526,...,0.638716,0.284236,0.178705,0.044497,-0.041057,-0.267453,-0.358968,-0.404977,-0.603138,0.757124
2,7325,0.518451,0.119400,-0.101666,-0.256064,-0.350055,-0.489664,-0.361839,-0.337958,-0.324172,...,-0.204770,-0.178232,-0.159118,-0.089595,0.036775,0.035835,0.603642,1.840441,0.114384,0.425406
3,389,0.504397,1.747607,0.082680,-0.510717,-0.447135,-0.388979,-0.175003,-0.017351,-0.218109,...,-0.286799,-0.526365,-0.761056,-0.716411,-0.746843,-0.672783,-0.646998,-0.678567,-0.779673,-0.692131
4,141994,-0.459337,-0.416012,-0.159974,-0.016193,0.064829,0.113438,0.080832,0.068900,0.090594,...,0.727373,0.590593,0.520914,0.243105,0.067435,-0.066206,-0.201791,-0.272793,-0.218129,-0.798766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,25833,-0.395722,0.248768,-0.155262,-0.638465,-0.557849,-0.619548,-0.418294,-0.375420,-0.391431,...,-0.417713,-0.417161,-0.546078,-0.435577,-0.455566,-0.419098,-0.286396,0.014118,-0.142986,-0.341553
2712,163653,1.790960,-0.794477,-0.832966,-0.610382,-0.465235,-0.402068,-0.176795,-0.073108,-0.010295,...,-0.656342,-0.729382,-0.802296,-0.413439,-0.577619,-0.628848,0.967629,7.815015,0.342486,0.069753
2713,3895,1.930659,-0.313100,0.860508,0.342770,-0.259086,-0.461808,-0.395891,-0.364965,-0.362975,...,-0.635627,-0.600389,-0.739997,-0.636715,-0.597077,-0.566490,-0.575555,-0.625625,-0.763762,2.473039
2714,77177,-0.646695,-0.944922,-0.937606,-0.732218,-0.555734,-0.422205,-0.391411,-0.330117,-0.259499,...,3.332403,1.190849,0.428781,0.391744,0.288546,1.171040,2.127649,-0.312756,0.177565,0.007668


,movieId,0,1,2,3,4,5,6,7,8,...,Mystery,News,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,8920,-0.550110,-0.742012,-0.837285,-0.631286,2.125147,1.712984,-0.156185,-0.289170,-0.325896,...,0,0,0,0,0,0,0,0,0,0
1,93040,1.940170,1.265333,-0.814315,-0.772337,-0.582296,-0.588000,-0.425014,-0.379776,-0.359526,...,0,0,0,0,0,0,0,0,1,0
2,7325,0.518451,0.119400,-0.101666,-0.256064,-0.350055,-0.489664,-0.361839,-0.337958,-0.324172,...,0,0,0,0,0,0,0,0,0,0
3,389,0.504397,1.747607,0.082680,-0.510717,-0.447135,-0.388979,-0.175003,-0.017351,-0.218109,...,0,0,1,0,0,0,0,0,0,0
4,141994,-0.459337,-0.416012,-0.159974,-0.016193,0.064829,0.113438,0.080832,0.068900,0.090594,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,25833,-0.395722,0.248768,-0.155262,-0.638465,-0.557849,-0.619548,-0.418294,-0.375420,-0.391431,...,0,0,1,0,0,0,0,0,0,0
2712,163653,1.790960,-0.794477,-0.832966,-0.610382,-0.465235,-0.402068,-0.176795,-0.073108,-0.010295,...,0,0,0,0,0,0,0,0,0,0
2713,3895,1.930659,-0.313100,0.860508,0.342770,-0.259086,-0.461808,-0.395891,-0.364965,-0.362975,...,1,0,0,0,0,0,0,0,0,0
2714,77177,-0.646695,-0.944922,-0.937606,-0.732218,-0.555734,-0.422205,-0.391411,-0.330117,-0.259499,...,0,0,0,0,0,0,0,0,0,0


### df_test_reduced:

In [768]:
df_test_preReduced_f = df_test_preReduced.drop('movieId', axis=1)
df_test_preReduced_f

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,-0.550110,-0.742012,-0.837285,-0.631286,2.125147,1.712984,-0.156185,-0.289170,-0.325896,-0.298186,...,-0.827856,-0.823194,-1.027804,-0.791680,-0.793423,-0.693096,-0.611653,-0.547066,1.182421,1.751948
1,1.940170,1.265333,-0.814315,-0.772337,-0.582296,-0.588000,-0.425014,-0.379776,-0.359526,-0.347788,...,0.638716,0.284236,0.178705,0.044497,-0.041057,-0.267453,-0.358968,-0.404977,-0.603138,0.757124
2,0.518451,0.119400,-0.101666,-0.256064,-0.350055,-0.489664,-0.361839,-0.337958,-0.324172,-0.277837,...,-0.204770,-0.178232,-0.159118,-0.089595,0.036775,0.035835,0.603642,1.840441,0.114384,0.425406
3,0.504397,1.747607,0.082680,-0.510717,-0.447135,-0.388979,-0.175003,-0.017351,-0.218109,-0.182450,...,-0.286799,-0.526365,-0.761056,-0.716411,-0.746843,-0.672783,-0.646998,-0.678567,-0.779673,-0.692131
4,-0.459337,-0.416012,-0.159974,-0.016193,0.064829,0.113438,0.080832,0.068900,0.090594,0.088449,...,0.727373,0.590593,0.520914,0.243105,0.067435,-0.066206,-0.201791,-0.272793,-0.218129,-0.798766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,-0.395722,0.248768,-0.155262,-0.638465,-0.557849,-0.619548,-0.418294,-0.375420,-0.391431,-0.342700,...,-0.417713,-0.417161,-0.546078,-0.435577,-0.455566,-0.419098,-0.286396,0.014118,-0.142986,-0.341553
2712,1.790960,-0.794477,-0.832966,-0.610382,-0.465235,-0.402068,-0.176795,-0.073108,-0.010295,-0.146839,...,-0.656342,-0.729382,-0.802296,-0.413439,-0.577619,-0.628848,0.967629,7.815015,0.342486,0.069753
2713,1.930659,-0.313100,0.860508,0.342770,-0.259086,-0.461808,-0.395891,-0.364965,-0.362975,-0.347788,...,-0.635627,-0.600389,-0.739997,-0.636715,-0.597077,-0.566490,-0.575555,-0.625625,-0.763762,2.473039
2714,-0.646695,-0.944922,-0.937606,-0.732218,-0.555734,-0.422205,-0.391411,-0.330117,-0.259499,-0.351603,...,3.332403,1.190849,0.428781,0.391744,0.288546,1.171040,2.127649,-0.312756,0.177565,0.007668


In [769]:
df_test_reduced = pca_umap_reduction(df_test_preReduced_f)
df_test_reduced

/Users/acerog/Documents/UTEC/Machine Learning/Projects/2/clustering-project-2/venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/acerog/Documents/UTEC/Machine Learning/Projects/2/clustering-project-2/venv/lib/python3.13/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


array([[3.5880003, 5.666133 , 4.5844927, ..., 4.2244306, 7.581219 ,
        4.88921  ],
       [3.0947728, 6.4841194, 2.743681 , ..., 3.842392 , 8.285978 ,
        4.868848 ],
       [3.2917583, 6.3098435, 2.857763 , ..., 3.9611313, 8.187398 ,
        4.9020753],
       ...,
       [4.7310443, 5.4765334, 3.8506024, ..., 4.2754297, 7.9320364,
        5.0799565],
       [1.9759432, 7.232176 , 3.839108 , ..., 4.1176624, 6.6526713,
        4.704371 ],
       [2.8767984, 6.3952937, 4.360308 , ..., 3.4038436, 8.157487 ,
        4.8944235]], shape=(2716, 10), dtype=float32)

In [770]:
df_test_reduced = pd.DataFrame(df_test_reduced)
df_test_reduced

,0,1,2,3,4,5,6,7,8,9
0,3.588000,5.666133,4.584493,8.579001,7.293003,4.570117,4.978430,4.224431,7.581219,4.889210
1,3.094773,6.484119,2.743681,8.063793,6.207491,5.014139,5.651404,3.842392,8.285978,4.868848
2,3.291758,6.309844,2.857763,8.182334,6.295832,4.908294,5.486188,3.961131,8.187398,4.902075
3,2.128102,6.702174,3.761693,8.866406,5.462176,5.356188,5.302478,4.168327,7.489252,4.439308
4,2.339074,6.758739,4.171293,8.395494,6.897200,5.739769,5.256337,4.410801,6.511393,4.702597
...,...,...,...,...,...,...,...,...,...,...
2711,2.262707,5.757225,4.229489,8.414571,6.095675,4.879224,5.331727,4.056153,7.950168,4.748795
2712,3.300639,7.040760,4.522184,7.376237,8.986684,5.432643,5.954186,3.508862,7.282051,4.942124
2713,4.731044,5.476533,3.850602,8.564969,7.528075,4.187242,4.840478,4.275430,7.932036,5.079957
2714,1.975943,7.232176,3.839108,7.998750,6.928601,6.034907,5.678324,4.117662,6.652671,4.704371


In [771]:
df_test_reduced["movieId"] = df_test_preReduced["movieId"]

cols = ['movieId'] + [col for col in df_test_reduced.columns if col != 'movieId']
df_test_reduced = df_test_reduced[cols]

display(df_test_reduced)

,movieId,0,1,2,3,4,5,6,7,8,9
0,8920,3.588000,5.666133,4.584493,8.579001,7.293003,4.570117,4.978430,4.224431,7.581219,4.889210
1,93040,3.094773,6.484119,2.743681,8.063793,6.207491,5.014139,5.651404,3.842392,8.285978,4.868848
2,7325,3.291758,6.309844,2.857763,8.182334,6.295832,4.908294,5.486188,3.961131,8.187398,4.902075
3,389,2.128102,6.702174,3.761693,8.866406,5.462176,5.356188,5.302478,4.168327,7.489252,4.439308
4,141994,2.339074,6.758739,4.171293,8.395494,6.897200,5.739769,5.256337,4.410801,6.511393,4.702597
...,...,...,...,...,...,...,...,...,...,...,...
2711,25833,2.262707,5.757225,4.229489,8.414571,6.095675,4.879224,5.331727,4.056153,7.950168,4.748795
2712,163653,3.300639,7.040760,4.522184,7.376237,8.986684,5.432643,5.954186,3.508862,7.282051,4.942124
2713,3895,4.731044,5.476533,3.850602,8.564969,7.528075,4.187242,4.840478,4.275430,7.932036,5.079957
2714,77177,1.975943,7.232176,3.839108,7.998750,6.928601,6.034907,5.678324,4.117662,6.652671,4.704371


In [773]:
df_test_reduced.to_csv('test_reduced.csv', index=False)

### df_test_reduced:

In [774]:
df_test_genere_preReduced_f = df_test_genere_preReduced.drop('movieId', axis=1)
df_test_genere_preReduced_f

,0,1,2,3,4,5,6,7,8,9,...,Mystery,News,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,-0.550110,-0.742012,-0.837285,-0.631286,2.125147,1.712984,-0.156185,-0.289170,-0.325896,-0.298186,...,0,0,0,0,0,0,0,0,0,0
1,1.940170,1.265333,-0.814315,-0.772337,-0.582296,-0.588000,-0.425014,-0.379776,-0.359526,-0.347788,...,0,0,0,0,0,0,0,0,1,0
2,0.518451,0.119400,-0.101666,-0.256064,-0.350055,-0.489664,-0.361839,-0.337958,-0.324172,-0.277837,...,0,0,0,0,0,0,0,0,0,0
3,0.504397,1.747607,0.082680,-0.510717,-0.447135,-0.388979,-0.175003,-0.017351,-0.218109,-0.182450,...,0,0,1,0,0,0,0,0,0,0
4,-0.459337,-0.416012,-0.159974,-0.016193,0.064829,0.113438,0.080832,0.068900,0.090594,0.088449,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,-0.395722,0.248768,-0.155262,-0.638465,-0.557849,-0.619548,-0.418294,-0.375420,-0.391431,-0.342700,...,0,0,1,0,0,0,0,0,0,0
2712,1.790960,-0.794477,-0.832966,-0.610382,-0.465235,-0.402068,-0.176795,-0.073108,-0.010295,-0.146839,...,0,0,0,0,0,0,0,0,0,0
2713,1.930659,-0.313100,0.860508,0.342770,-0.259086,-0.461808,-0.395891,-0.364965,-0.362975,-0.347788,...,1,0,0,0,0,0,0,0,0,0
2714,-0.646695,-0.944922,-0.937606,-0.732218,-0.555734,-0.422205,-0.391411,-0.330117,-0.259499,-0.351603,...,0,0,0,0,0,0,0,0,0,0


In [775]:
df_test_genere_reduced = pca_umap_reduction(df_test_genere_preReduced_f)
df_test_genere_reduced

/Users/acerog/Documents/UTEC/Machine Learning/Projects/2/clustering-project-2/venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/acerog/Documents/UTEC/Machine Learning/Projects/2/clustering-project-2/venv/lib/python3.13/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


array([[3.7323236, 5.484754 , 4.2985506, ..., 4.650095 , 6.4560485,
        5.543898 ],
       [3.743076 , 6.1194444, 2.0912247, ..., 4.0757036, 7.403516 ,
        5.452233 ],
       [3.8863587, 6.0195656, 2.4280293, ..., 4.170857 , 7.282488 ,
        5.47116  ],
       ...,
       [4.965521 , 5.1237473, 3.796161 , ..., 4.8156605, 6.549072 ,
        5.5976915],
       [2.2904472, 7.113819 , 2.9424684, ..., 4.277075 , 6.5271764,
        5.0486965],
       [3.0324855, 5.9441757, 3.792851 , ..., 3.570171 , 7.870109 ,
        5.588693 ]], shape=(2716, 10), dtype=float32)

In [776]:
df_test_genere_reduced = pd.DataFrame(df_test_genere_reduced)
df_test_genere_reduced

,0,1,2,3,4,5,6,7,8,9
0,3.732324,5.484754,4.298551,9.189777,7.382783,5.247823,4.874993,4.650095,6.456048,5.543898
1,3.743076,6.119444,2.091225,8.620902,6.440711,4.729836,5.184028,4.075704,7.403516,5.452233
2,3.886359,6.019566,2.428029,8.584360,6.758828,4.842232,5.149366,4.170857,7.282488,5.471160
3,2.991841,6.843195,2.755834,9.681257,5.664947,5.238273,5.175094,4.221187,6.790326,5.426101
4,2.539688,6.828892,3.292749,8.749533,6.564216,6.098940,5.401007,4.559855,6.166941,5.079424
...,...,...,...,...,...,...,...,...,...,...
2711,2.839093,5.835024,3.528517,9.468704,6.200972,4.945968,4.982979,4.113078,7.142540,5.592292
2712,2.970801,6.469445,3.970148,7.295586,8.425855,6.715026,5.645701,3.852214,7.209468,5.297081
2713,4.965521,5.123747,3.796161,8.774676,7.994304,4.805124,4.774508,4.815660,6.549072,5.597692
2714,2.290447,7.113819,2.942468,8.371104,6.497417,6.266715,5.597289,4.277075,6.527176,5.048697


In [777]:
df_test_genere_reduced["movieId"] = df_test_genere_preReduced["movieId"]

cols = ['movieId'] + [col for col in df_test_genere_reduced.columns if col != 'movieId']
df_test_genere_reduced = df_test_genere_reduced[cols]

display(df_test_genere_reduced)

,movieId,0,1,2,3,4,5,6,7,8,9
0,8920,3.732324,5.484754,4.298551,9.189777,7.382783,5.247823,4.874993,4.650095,6.456048,5.543898
1,93040,3.743076,6.119444,2.091225,8.620902,6.440711,4.729836,5.184028,4.075704,7.403516,5.452233
2,7325,3.886359,6.019566,2.428029,8.584360,6.758828,4.842232,5.149366,4.170857,7.282488,5.471160
3,389,2.991841,6.843195,2.755834,9.681257,5.664947,5.238273,5.175094,4.221187,6.790326,5.426101
4,141994,2.539688,6.828892,3.292749,8.749533,6.564216,6.098940,5.401007,4.559855,6.166941,5.079424
...,...,...,...,...,...,...,...,...,...,...,...
2711,25833,2.839093,5.835024,3.528517,9.468704,6.200972,4.945968,4.982979,4.113078,7.142540,5.592292
2712,163653,2.970801,6.469445,3.970148,7.295586,8.425855,6.715026,5.645701,3.852214,7.209468,5.297081
2713,3895,4.965521,5.123747,3.796161,8.774676,7.994304,4.805124,4.774508,4.815660,6.549072,5.597692
2714,77177,2.290447,7.113819,2.942468,8.371104,6.497417,6.266715,5.597289,4.277075,6.527176,5.048697


In [778]:
df_test_genere_reduced.to_csv('test_genere_reduced.csv', index=False)

### Reduccion de la Dimencionalidad de df_test_complete_f

In [884]:
df_test_complete_f_noId = df_test_complete_f.drop('movieId', axis=1)

In [885]:
df_test_complete_reduced = pca_umap_reduction(df_test_complete_f_noId)
df_test_complete_reduced

/Users/acerog/Documents/UTEC/Machine Learning/Projects/2/clustering-project-2/venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/acerog/Documents/UTEC/Machine Learning/Projects/2/clustering-project-2/venv/lib/python3.13/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


array([[7.810063 , 3.3885505, 7.01981  , ..., 3.1097524, 3.1269717,
        4.51762  ],
       [6.236599 , 4.4753666, 6.230341 , ..., 3.4411266, 2.9869177,
        4.273797 ],
       [7.180733 , 4.0589976, 8.02827  , ..., 2.621932 , 2.6854508,
        4.5202928],
       ...,
       [6.6894107, 3.2147646, 6.823566 , ..., 5.1517086, 3.8408265,
        5.252277 ],
       [7.3538027, 3.933844 , 7.855059 , ..., 3.0374823, 2.7669773,
        4.5013876],
       [5.3729825, 4.88594  , 7.455801 , ..., 3.3465877, 2.447756 ,
        4.2412252]], shape=(2908, 10), dtype=float32)

In [886]:
df_test_complete_reduced = pd.DataFrame(df_test_complete_reduced)
df_test_complete_reduced

,0,1,2,3,4,5,6,7,8,9
0,7.810063,3.388551,7.019810,0.984415,4.032607,5.533764,5.052744,3.109752,3.126972,4.517620
1,6.236599,4.475367,6.230341,1.939883,2.475873,5.386091,5.772178,3.441127,2.986918,4.273797
2,7.180733,4.058998,8.028270,2.430593,3.652376,5.642815,5.043956,2.621932,2.685451,4.520293
3,7.441164,3.126659,6.508239,1.915993,3.043106,4.548501,4.765089,3.583521,3.718896,4.988709
4,6.884728,4.074530,7.807277,2.501562,3.381725,5.506544,5.103591,2.615122,2.721788,4.518814
...,...,...,...,...,...,...,...,...,...,...
2903,7.987535,4.140912,6.301332,1.126496,3.427325,5.711436,5.596533,4.792279,3.540698,4.596154
2904,7.878825,4.406694,5.746078,1.241730,3.222372,5.281231,5.376145,3.221180,3.268605,4.190710
2905,6.689411,3.214765,6.823566,2.039755,2.684155,5.014174,5.326220,5.151709,3.840827,5.252277
2906,7.353803,3.933844,7.855059,1.873578,3.860351,5.851604,5.187011,3.037482,2.766977,4.501388


In [887]:
df_test_complete_reduced["movieId"] = df_test_complete_f["movieId"]

cols = ['movieId'] + [col for col in df_test_complete_reduced.columns if col != 'movieId']
df_test_complete_reduced = df_test_complete_reduced[cols]

display(df_test_complete_reduced)

,movieId,0,1,2,3,4,5,6,7,8,9
0,389,7.810063,3.388551,7.019810,0.984415,4.032607,5.533764,5.052744,3.109752,3.126972,4.517620
1,8920,6.236599,4.475367,6.230341,1.939883,2.475873,5.386091,5.772178,3.441127,2.986918,4.273797
2,93040,7.180733,4.058998,8.028270,2.430593,3.652376,5.642815,5.043956,2.621932,2.685451,4.520293
3,141994,7.441164,3.126659,6.508239,1.915993,3.043106,4.548501,4.765089,3.583521,3.718896,4.988709
4,7325,6.884728,4.074530,7.807277,2.501562,3.381725,5.506544,5.103591,2.615122,2.721788,4.518814
...,...,...,...,...,...,...,...,...,...,...,...
2903,180297,7.987535,4.140912,6.301332,1.126496,3.427325,5.711436,5.596533,4.792279,3.540698,4.596154
2904,165639,7.878825,4.406694,5.746078,1.241730,3.222372,5.281231,5.376145,3.221180,3.268605,4.190710
2905,27074,6.689411,3.214765,6.823566,2.039755,2.684155,5.014174,5.326220,5.151709,3.840827,5.252277
2906,95149,7.353803,3.933844,7.855059,1.873578,3.860351,5.851604,5.187011,3.037482,2.766977,4.501388


In [888]:
df_test_complete_reduced.to_csv('test_complete_reduced.csv', index=False)